## FFS

In [ ]:
# ARCFOFO lattice definition
from atpy import *
#---ELEMENTS DIFINE---#
MK0 = Marker(beta_x=0.04,alpha_x=0,gamma_x=25,beta_y=0.0008,alpha_y=0,gamma_y=1250)
L1 = Drift(l=0.935)
L2 = Drift(l=0.2)
QD1 = Quadrupole(l=0.3,k1=-5.734671860546432)
QF2 = Quadrupole(l=0.4,k1= 2.8060832378782052)
L3 = Drift(l=6)
L31 = Drift(l=4.5)
L32 = Drift(l=0.5)
QD3 = Quadrupole(l=0.4,k1=-1.6923916161556376)
L4 = Drift(l=0.22)
QF4 = Quadrupole(l=0.4,k1= 1.705134294804317)
L51 = Drift(l=0.5)
L52 = Drift(l=1.5-0.0)

QD5 = Quadrupole(l=0.4,k1=-2.1692589345478774)
L6 = Drift(l=0.2)
L61 = Drift(l=1.5)
L62 = Drift(l=1.0)
QF6 = Quadrupole(l=0.4,k1= -1.663370351531717+0.5)
L7 = Drift(l=1.0)
L71 = Drift(l=1.0)
L72 = Drift(l=1.0)


QD7 = Quadrupole(l=0.4,k1=-1.3)
L8 = Drift(l=0.8)
L81 = Drift(l=0.5+0.5)
L82 = Drift(l=0.5+0.5)
QF8 = Quadrupole(l=0.4,k1= 1.6)
L9 = Drift(l=1.0)

QD9 = Quadrupole(l=0.4,k1= -0.6)
L10= Drift(l=1.0)

B1 = Dipole(l=1,angle=0.05)
B2 = Dipole(l=1,angle=0.05)

#---CELL DEFINE---#
ARCFODO=Lattice(MK0,L1,QD1,L2,QF2,L31,B1,L32,QD3,L4,QF4,L51)#,L52)#,QD5,L6,QF6,L7,QD7,L81,L82,QF8,L9,QD9,L10,L10,QD9,L9,QF8,L82,L81)

#---OPTIMIZATION---#
var= (#{'place':0,'variables':{'beta_x':(0.03,0.045)}},#,'beta_y':(0.00075,0.00085)
#       {'place':1,'variables':{'l':(0.9,0.99)}},
      {'place':2,'variables':{'k1':(-9.2, -3.1)}},#
#       {'place':3,'variables':{'l':(0.1,1.0/2)}},
      {'place':4,'variables':{'k1':(2.01, 8)}},#
      {'place':5,'variables':{'l':(2.5,7.5)}},
      {'place':6+2,'variables':{'k1':(-4, 0)}},#
#       {'place':7+2,'variables':{'l':(0.1,1)}},
      {'place':8+2,'variables':{'k1':(0.01, 4)}},
      {'place':7+4,'variables':{'l':(0.1,2)}},
     )

CV= (
#     {'place':2 ,'constraints':{'beta_y':(200,2000)}},
#      {'place':7 ,'constraints':{'alpha_x':(1,600),'alpha_y':(1,600)} },
     {'place':11 ,'constraints':{'alpha_x':0,'alpha_y':0,"nu_x":0.5,"nu_y":0.5}},#'beta_x':(0,10),'beta_y':(0,10),
#      {'place':9+2 ,'constraints':{'alpha_x':0,'alpha_y':0} },
#      {'place':(0,9+2) ,'constraints':{'telescope_x':-15,'telescope_y':-15}},
#      {'place':9+2 ,'constraints':{'beta_x':(0,4),'beta_y':(0,0.8)}},
     
#      {'place':18 ,'constraints':{'alpha_x':0,'alpha_y':0,'beta_y':(0,0.01)} },
#      {'place':23 ,'constraints':{'alpha_x':0,'alpha_y':0,'beta_x':(0,0.01)} },
#      {'place':(5,11) ,'constraints':{'beta_x':(0,400),'beta_y':(0,450)}},
#      {'place':16 ,'constraints':{'beta_x':(0,30),'beta_y':(0,20)}},
#      ,'beta_x':(20,150),'alpha_x':0,'alpha_y':0,'eta_x':0,'
    )

opti = ({'place':4,'optima':{'beta_x':1}},
        {'place':4,'optima':{'beta_y':1}},
#         {'place':8,'optima':{'alpha_y':-1}},
#         {'place':8,'optima':{'alpha_x':-1}},
       )

variables = Variables(*var)
constraints = Constraints(*CV)
optima = Optima(*opti)

ARCFODO.set_variables(variables)
ARCFODO.set_constraints(constraints)
ARCFODO.set_optima(optima)




from myproblem import MyProblem as Problem
from main import main
from cProfile import run
%matplotlib inline
match =False
problem = Problem(ARCFODO)
NDSet,pop_trace = main(problem,NIND=300,MAXGEN=100, CR=0.8,drawing=1, Parallel=True)

# run('main(problem,NIND=400)',sort="cumulative")

#

In [ ]:
%%cython -a -3 -+
# cython: cdivision=True, boundscheck=False,
from atpy cimport Element,Marker,Drift, Dipole, Quadrupole
from atpy cimport Variables
from atpy cimport Constraints


from atpy cimport Optima

from atpy cimport CppElement, CppMarker, CppDrift, CppDipole, CppQuadrupole

from libc.math cimport fmax,fabs,pi
from libc.math cimport sqrt as cysqrt
from libc.string cimport memcpy
from cymem.cymem cimport Pool
cimport cython
import numpy as np

cdef struct Matrix:
    double[6][6] r


cdef class Lattice:
    '''
    需要修改相应myproblem.py
    '''
    cdef:
        Pool mem
        CppElement** elems
        double**     parms
        double**     twiss
        Matrix*         loc_properties
        double[8]       rad_integrals
        #natural emittance, Synchrotron energy loss U0 [GeV], momentum compact factor alpha, 
        double[8]       global_parameters
        Variables    variables
        Constraints  constraints
        Optima       optima
        bint     cell
        int      nseq
        
        double          circumference
        double          energy
        double          mass
        
        dict            parameters2index
        list            parameters
        readonly list         pyseq, pyelems
    
    
    def __cinit__(self,*args, bint cell=True):
        cdef Element temp_elem
        self.mem = Pool()
        self.cell = cell
        
        self.energy = 2500
        self.mass   = 0.511
        
        self.parameters2index = {'l':0, 'angle':1, 'k1':2, 'k2':3, 'e1':4, 'e2':5,
                            'beta_x':0,'alpha_x':1,'gamma_x':2,'beta_y':3,'alpha_y':4,'gamma_y':5,'beta_z':6,'alpha_z':7,'gamma_z':8,
                            'nu_x':9,'nu_y':10,'nu_z':11,'eta_x':12,'etap_x':13,'eta_y':14,'etap_y':15,'eta_z':16,'etap_z':17,
                            'emit':0,'chromx':6,'chromy':7}
        self.parameters = ['l','angle','k1','k2','e1','e2','beta_x','alpha_x','beta_y','alpha_y','beta_z','alpha_z',
                                    'eta_x','etap_x','eta_y','etap_y','eta_z','etap_z','nu_x','nu_y','nu_z','emit','chromx','chromy']
        self.pyseq = []
        self.pyelems = []
        self.nseq = len(args)
        
        self.twiss =<double**>self.mem.alloc(self.nseq,sizeof(double*))
        self.parms =<double**>self.mem.alloc(self.nseq,sizeof(double*))
        self.elems =<CppElement**>self.mem.alloc(self.nseq,sizeof(CppElement*))
        if self.cell==True:
            self.loc_properties = <Matrix*>self.mem.alloc(self.nseq,sizeof(Matrix) )
        
        assert all([isinstance(arg,Element) for arg in args]), 'Args must be Element class or its subclasses'
        for index,elem in enumerate(args):
            temp_elem = elem if elem not in self.pyelems else elem.copy()
            self.pyseq.append(temp_elem)
            if elem not in self.pyelems:
                self.pyelems.append(temp_elem)
            self.twiss[index] = (<Element?>self.pyseq[index]).bind2twiss()
            self.elems[index] = (<Element?>self.pyseq[index]).bind2element()
            self.parms[index] = (<Element?>self.pyseq[index]).bind2parameter()
            

    
    cdef void _matmul(self,double* returned, double* lm, double* rm)nogil:
        cdef int i,j,k
        for i in range(6):
            for j in range(6):
                returned[6*i+j] = (lm[6*i]*rm[j] + lm[6*i+1]*rm[6+j] + lm[6*i+2]*rm[12+j] 
                                    + lm[6*i+3]*rm[18+j] + lm[6*i+4]*rm[24+j] + lm[6*i+5]*rm[30+j] )
    
    
    cdef void _init_twiss(self,double* transfermatrix)nogil:
        cdef double[6][6] r66
        cdef double cscmux,cscmuy,cosmux,cosmuy
        memcpy(&r66[0][0], transfermatrix, sizeof(r66))
        if self.cell==True and fabs(r66[0][0] + r66[1][1]) <2.0 and fabs(r66[2][2] + r66[3][3]) <2.0:
            cosmux = 0.5*(r66[0][0] + r66[1][1])
            cosmuy = 0.5*(r66[2][2] + r66[3][3])
            cscmux = r66[0][1]/( fabs(r66[0][1])*cysqrt(1-cosmux*cosmux) ) #1/sin(mux)
            cscmuy = r66[2][3]/( fabs(r66[2][3])*cysqrt(1-cosmuy*cosmuy) ) #1/sin(muy)
            self.twiss[0][0] = r66[0][1]*cscmux
            self.twiss[0][1] = 0.5*(r66[0][0]-r66[1][1])*cscmux
            self.twiss[0][2] = -r66[1][0]*cscmux
            self.twiss[0][3] = r66[2][3]*cscmuy
            self.twiss[0][4] = 0.5*(r66[2][2]-r66[3][3])*cscmuy
            self.twiss[0][5] = -r66[3][2]*cscmuy
            self.twiss[0][12]= 0.5*(r66[0][5]*(1.0-r66[1][1]) + r66[0][1]*r66[1][5])/(1.0-cosmux)
            self.twiss[0][13]= 0.5*(r66[1][5]*(1.0-r66[0][0]) + r66[1][0]*r66[0][5])/(1.0-cosmux)
    
    
    cdef void _evaluate_lattice(self)nogil:
        cdef int i,j,k,p, n_integral = 8
        self.circumference = 0.0
        cdef double[:,:] r66
        for i in range(8):
            self.rad_integrals[i] = 0
            self.global_parameters[i]=0
        if self.cell==True:
            memcpy( self.loc_properties[0].r, self.elems[0].M, sizeof(self.elems[0].M) )
            for i in range(1,self.nseq):
                self._matmul(&self.loc_properties[i].r[0][0], &self.elems[i].M[0][0], &self.loc_properties[i-1].r[0][0])
        
            self._init_twiss(&self.loc_properties[self.nseq-1].r[0][0])
        
        for i in range(1,self.nseq):
            self.elems[i].get_twiss(self.twiss[i-1],self.twiss[i])
            self.elems[i].get_dispersion(self.twiss[i-1],self.twiss[i])
            self.elems[i].get_phase(self.twiss[i-1],self.twiss[i])
            self.elems[i].get_rad_integral(self.twiss[i-1], &self.rad_integrals[0])
            self.elems[i].get_chrom(self.twiss[i-1],self.twiss[i])
            self.circumference += self.parms[i][0]
            #total horizontal chromaticity
            self.global_parameters[6] += self.twiss[i][18]
            #total vertical chromaticity
            self.global_parameters[7] += self.twiss[i][19]
        self._get_global_parameters()
    
    cdef void _get_global_parameters(self)nogil:
        cdef double* I = &self.rad_integrals[0]
        cdef:
            double Cq = 3.83E-13
            double E0 = self.energy
            double gamma0 = self.energy/self.mass
        #natural emittance
        self.global_parameters[0] = Cq*gamma0*gamma0*I[4]/(I[1] - I[3])
        #momentum compaction factor
        self.global_parameters[1] = I[0]/self.circumference
        #energy loss U0 [GeV]
        self.global_parameters[2] = 1.404*E0**4*I[1]
        #square of Energy dispersion
        self.global_parameters[3] = Cq*gamma0*gamma0*I[2]/(2*I[1] + I[3])
        #spin polarization I[6]:I3a in S.Y. Lee's book
        self.global_parameters[4] = -1.6/cysqrt(3)*I[6]/I[2]
        #damping factor D Jx=1-D,Jy=1,Jz=2+D
        self.global_parameters[5] = I[3]/I[1]
        
        
    cdef void update_variables(self, double[:] variables, int num_variables)nogil:
        cdef int i, j
        for j in range(num_variables):
            if self.variables.index[0][j]==1:
                self.twiss[0][ self.variables.index[1][j] ] = variables[j]
            elif self.variables.index[0][j]==0:
                self.parms[ self.variables.index[2][j] ][ self.variables.index[1][j] ] = variables[j]
        self.twiss[0][2] = (1 + self.twiss[0][1]**2)/self.twiss[0][0]
        self.twiss[0][5] = (1 + self.twiss[0][4]**2)/self.twiss[0][3]
        for j in self.variables.indexset:
            self.elems[j].update( &self.parms[j][0] )
    
    
    cdef void update_constraints(self, double[:] CV, int num_constraints, double[:] scale)nogil:
        cdef int i, j, k, l, index, start,end, sizeofmatrix = sizeof(self.elems[0].M)
        cdef double lower, upper,temp_max
        for j in range(num_constraints):
            start = self.constraints.index[2][j] 
            end = self.constraints.index[3][j] 
            lower = self.constraints.bounds[0][j] 
            upper = self.constraints.bounds[1][j]
            index = self.constraints.index[1][j]
            if self.constraints.index[0][j]==0:
                #phase advance or twiss differnce between two position
                #CV[j] = fmax(0.0, fabs(self.twiss[ end ][index] - self.twiss[ start ][index] - (upper + lower)/2.0)-(upper - lower)/2.0 )
                CV[j] = scale[j]*fmax(0.0, fabs(self.twiss[ end ][index] - self.twiss[ start ][index] - 0.5*(upper + lower))-0.5*(upper - lower) )
            elif self.constraints.index[0][j]==1:
                temp_max = -1.0e8
                if end > start:
                    for i in range(start,end+1):
                        temp_max = fmax(temp_max, fabs(self.twiss[i][index] - (upper + lower)/2.0)-(upper - lower)/2.0)
                else:
                    temp_max = fmax(0.0, fabs(self.twiss[start][index] - (upper + lower)/2.0)-(upper - lower)/2.0 )
                CV[j] = scale[j]*fmax(0.0, temp_max)
            #elif self.constraints.index[0][j]==1:
                #twiss parameter at start position
            #    CV[j] = scale[j]*fmax(0.0, fabs(self.twiss[start][index] - 0.5*(upper + lower))-0.5*(upper - lower))
            elif self.constraints.index[0][j]==2:
                #global parameters such as emit, U0...
                CV[j] = scale[j]*fmax(0.0, fabs(self.global_parameters[index] - 0.5*(upper + lower))-0.5*(upper - lower) )
        if self.cell==True:
            CV[num_constraints] = (fmax(0.0,fabs( self.loc_properties[self.nseq].r[0][0] + self.loc_properties[self.nseq].r[1][1] ) - 2.0) 
                                   +fmax(0.0,fabs( self.loc_properties[self.nseq].r[2][2] + self.loc_properties[self.nseq].r[3][3] ) - 2.0) )
            #if CV[num_constraints]> 1.0e-20:
            #    for j in range(num_constraints):
            #        CV[j] = 1.0e8

    

    cdef void update_optima(self, double[:] objectives, int num_optima)nogil:
        cdef int i, j, start,end,index
        for j in range(num_optima):
            index = self.optima.index[1][j]
            start = self.optima.index[2][j]
            end = self.optima.index[3][j]
            
            if self.optima.index[0][j]==0:
                objectives[j]= (self.twiss[ end ][ index ] - self.twiss[ start ][ index ]) #only the twiss parameters nu
            elif self.optima.index[0][j]==1:
                if end == 0:
                    objectives[j]= self.parms[ start ][ index ]
                if end == 1:
                    objectives[j]= self.twiss[ start ][ index ]
            elif self.optima.index[0][j]==2:
                objectives[j] = fabs( self.global_parameters[index] )
    
    
    def moea_without_CV(self, double[:,:] variables, double[:,:] objectives, double[:] scale, double factor=1.0e8, bint addCV=False):
        '''
        moea_with_CV(double[:,:] variables, double[:,:] objectives, double[:] scale)
        '''
        cdef:
            int i, num_variables= variables.shape[1], num_optima = objectives.shape[1], num_pop = variables.shape[0]
            int NCV = scale.shape[0]
            double totalCV = 0.0
        cdef double[:] CV = np.zeros( NCV , dtype='float')
        
        assert num_variables == self.variables.index[0].size(), "Input numpy array doesn't match the variables!"
        if addCV is True:
            assert num_optima == self.optima.index[0].size(), "Input numpy array doesn't match the optima!"
        else:
            assert num_optima == self.optima.index[0].size()+1, "Input numpy array doesn't match the optima!"
        for i in range(num_pop):
            self.update_variables(variables[i,:], num_variables)
            self._evaluate_lattice()
            self.update_constraints(CV, NCV-1, scale)
            self.update_optima(objectives[i,:], self.optima.index[0].size())
            totalCV = 0.0
            for j in range(NCV):
                totalCV += CV[j]
            if addCV is True:
                for j in range(num_optima):
                    objectives[i,j]+=factor*totalCV
            elif addCV is False:
                objectives[i,num_optima-1]=totalCV
            
    
    
    def get_results(self,double[:,:] variables, double[:,:] objectives, double[:,:] CV, double[:] scale):
        '''
        get_results(variables, objectives, CV)
        '''
        cdef int i, num_variables= variables.shape[1], num_optima = objectives.shape[1], num_constraints=CV.shape[1], num_pop = variables.shape[0]
        
        assert num_variables == self.variables.index[0].size(), "Input numpy array doesn't match the variables!"
        assert num_optima == self.optima.index[0].size(), "Input numpy array doesn't match the optima!"
        if self.cell==True:
            assert num_constraints == self.constraints.index[0].size()+1 , "Input numpy array: {} doesn't match the constraints\
            :{}!".format(num_constraints,self.constraints.index[0].size())
        else:
            assert num_constraints == self.constraints.index[0].size() , "Input numpy array: {} doesn't match the constraints\
            :{}!".format(num_constraints,self.constraints.index[0].size())
        for i in range(num_pop):
            self.update_variables(variables[i,:], num_variables)
            self._evaluate_lattice()
            self.update_constraints(CV[i,:], self.constraints.index[0].size(), scale)
            self.update_optima(objectives[i,:], num_optima)
             
    
    def evaluate_lattice(self):
        self._evaluate_lattice()
    
    
    cdef double _getitem(self,name,index=0):
        cdef int kind = self.parameters2index[name]
        if name in self.parameters[:6]:
            return self.parms[index][kind]
        elif name in self.parameters[6:21]:
            return self.twiss[index][kind]
        elif name in self.parameters[21:]:
            return self.global_parameters[kind]
    
    def __getitem__(self,arg):
        cdef int column, row
        if arg == 'cell':
            return self.cell
        elif type(arg) is dict:
            assert len(arg.keys())==1,'ArgsError:argument only receive dict with one key-value!'
            for key in arg.keys():
                position = key
            parameters=arg[key]
            assert type( position ) is tuple or type( position ) is int, \
            'key for the dict is tuple or int which are both index of element in the beamline!'
            assert type( parameters ) is list or type( parameters ) is str, \
            'value for the dict is list or str which are both name of the local parameters!'
            (row,position) = (len(position),position) if type(position) is tuple else (1,[position])
            (column,parameters) = (len(parameters),parameters) if type(parameters) is list else (1,[parameters])
            
            return_array = np.zeros((row,column))
            for i,value in enumerate(position):
                for j, name in enumerate(parameters):
                    
                    return_array[i,j]=self._getitem(name,value)
            return return_array
            
        elif type(arg) is list or type(arg) is tuple:
            return_array = np.zeros(len(arg))
            for index,name in enumerate(arg):
                assert type(name) is str,'arg must be the same types:like (str,str,str)'
                return_array[index] = self._getitem(name,self.nseq-1)
            return return_array
        elif type(arg) is str:
            if arg == 'variables':
                return self.variables
            elif arg == 'constraints':
                return self.constraints
            elif arg == 'optima':
                return self.optima
            elif arg == 'elements':
                return self.elements
            elif arg in self.parameters[21:]:
                return self._getitem(arg)
            else:
                raise ValueError('Error str delivered to __getitem__()!')
    
    
    def __setitem__(self, index, value):
        if index == 'variables':
            assert isinstance(value,Variables),'Wrong type is given for variables!'
            self.variables = value
        elif index == 'constraints':
            assert isinstance(value,Constraints),'Wrong type is given for constraints!'
            self.constraints = value
        elif index == 'optima':
            assert isinstance(value,Optima),'Wrong type is given for optima!'
            self.optima = value
        
        
    def __neg__(self):
        cdef list args = self.pyseq[::-1]
        cls = self.__class__
        return cls.__new__(cls,*args )
    
    
    def __add__(self, Lattice other not None):
        cdef list args = self.pyseq + other.pyseq
        cls = self.__class__
        return cls.__new__(cls, *args)
    
    
    def write2file(self, str filename, str filetype='SAD'):
        cdef dict code2kind = {0:'MARK ', 1:'DRIFT', 2:'BEND ', 4:'QUAD ', 6:'SEXT ', 8:'OCTU '}
        cdef dict namehead = {0:'M', 1:'D', 2:'B', 4:'Q', 6:'S', 8:'O'}
        cdef dict index2parms={0:'L', 1:'ANGLE', 2:'K1', 3:'K2', 4:'E1', 5:'E2'}
        cdef double tol = 1.0e-8,value
        cdef int i, index, code, nparms=6
        cdef str elemstr
        cdef Element elem
        print('File name is : ', filename)
        with open(filename,'w+') as fn:
            for index,elem in enumerate(self.pyelems):
                code = elem.elem.elem_type
                elemstr= code2kind[ code ] + '    {}{:<4} = ('.format(namehead[code], index)
                for i in range(nparms):
                    if fabs(elem.parms[i]) > tol:
                        value = elem.parms[i]*elem.parms[0] if i==2 or i==3 else elem.parms[i]
                        elemstr = elemstr + '{:<5}={:.6e}  '.format(index2parms[i], value)
                elemstr = elemstr + ');\n'
                fn.write(elemstr)
            # elemstr = 'LINE    LINE1 = ('
            # for elem in self.pyseq:
            #     elemstr = elemstr + elem.name + ' '
            # elemstr = elemstr + ');\n'
            # fn.write(elemstr)
        


In [ ]:
from geatpy import ndsort
ndsort?

## IR

In [ ]:
#STCF IR
from atpy import *
MK0        =     Marker('MK0'     ,beta_x=0.04,gamma_x=25,beta_y=0.0006,gamma_y=5000/3)
L1         =      Drift('L1'      ,l    =9.000000e-01  )
QD1        = Quadrupole('QD1'     ,l    =3.000000e-01  ,k1   =-6.660540e+00  )
L2         =      Drift('L2'      ,l    =1.765600e-01  )
QF2        = Quadrupole('QF2'     ,l    =4.000000e-01  ,k1   =3.525161e+00  )
L31        =      Drift('L31'     ,l    =7.492411e+00  )
B1         =     Dipole('B1'      ,l    =8.000000e-01  ,angle=2.000000e-02  )
L32        =      Drift('L32'     ,l    =5.000000e-01  )
QD3        = Quadrupole('QD3'     ,l    =4.000000e-01  ,k1   =-1.299573e+00  )
L4         =      Drift('L4'      ,l    =2.681480e-01  )
QF4        = Quadrupole('QF4'     ,l    =4.000000e-01  ,k1   =1.000000e-04  )
L51        =      Drift('L51'     ,l    =1.023210e+00  )
L52        =      Drift('L52'     ,l    =7.161780e-01  )
QD5        = Quadrupole('QD5'     ,l    =4.000000e-01  ,k1   =-4.302885e+00  )
L6         =      Drift('L6'      ,l    =2.000000e-01  )
QF6        = Quadrupole('QF6'     ,l    =4.000000e-01  ,k1   =3.609662e+00  )
L71        =      Drift('L71'     ,l    =4.535590e-01  )
B2         =     Dipole('B2'      ,l    =5.000000e-01  ,angle=4.965300e-02  )
L72        =      Drift('L72'     ,l    =1.127190e+00  )
QD7        = Quadrupole('QD7'     ,l    =4.000000e-01  ,k1   =-1.605968e+00  )
L81        =      Drift('L81'     ,l    =2.000000e-01  )
LSCY1      =      Drift('LSCY1'   ,l    =2.000000e-01  )
L82        =      Drift('L82'     ,l    =5.430200e-01  )
QF8        = Quadrupole('QF8'     ,l    =4.000000e-01  ,k1   =4.394400e-01  )
L91        =      Drift('L91'     ,l    =5.452170e-01  )
B3         =     Dipole('B3'      ,l    =5.000000e-01  ,angle=4.996300e-02  )
L92        =      Drift('L92'     ,l    =9.130450e-01  )
QF9        = Quadrupole('QF9'     ,l    =2.000000e-01  ,k1   =1.263098e+00  )
L10        =      Drift('L10'     ,l    =1.765512e+00  )
QF10       = Quadrupole('QF10'    ,l    =4.000000e-01  ,k1   =1.567516e+00  )
L102       =      Drift('L102'    ,l    =9.997691e-01  )
B7         =     Dipole('B7'      ,l    =5.000000e-01  ,angle=4.999900e-02  )
L103       =      Drift('L103'    ,l    =1.001069e-01  )
QF102      = Quadrupole('QF102'   ,l    =4.000000e-01  ,k1   =2.214806e-01  )
L111       =      Drift('L111'    ,l    =8.227606e-01  )
B8         =     Dipole('B8'      ,l    =5.000000e-01  ,angle=4.999996e-02  )
L112       =      Drift('L112'    ,l    =1.981832e+00  )
QD11       = Quadrupole('QD11'    ,l    =4.000000e-01  ,k1   =-1.487831e+00  )
L12        =      Drift('L12'     ,l    =2.667860e+00  )
QF12       = Quadrupole('QF12'    ,l    =4.000000e-01  ,k1   =1.514376e+00  )
L131       =      Drift('L131'    ,l    =1.000000e-01  )
LSCX1      =      Drift('LSCX1'   ,l    =2.000000e-01  )
L132       =      Drift('L132'    ,l    =3.931733e-01  )
QD13       = Quadrupole('QD13'    ,l    =4.000000e-01  ,k1   =-3.703105e-04  )
L141       =      Drift('L141'    ,l    =1.000000e-01  )
B5         =     Dipole('B5'      ,l    =5.000000e-01  ,angle=4.947350e-02  )
L142       =      Drift('L142'    ,l    =8.850312e-01  )
QD14       = Quadrupole('QD14'    ,l    =2.000000e-01  ,k1   =-1.411714e+00  )
B6         =     Dipole('B6'      ,l    =5.000000e-01  ,angle=1.251087e-03  )
L151       =      Drift('L151'    ,l    =1.164082e+00  )
QD19       = Quadrupole('QD19'    ,l    =4.000000e-01  ,k1   =-1.975741e+00  )
L18        =      Drift('L18'     ,l    =2.000000e-01  )
QF20       = Quadrupole('QF20'    ,l    =4.000000e-01  ,k1   =1.320857e+00  )
L19        =      Drift('L19'     ,l    =6.411217e-01  )
B9         =     Dipole('B9'      ,l    =5.000000e-01  ,angle=4.035181e-02  )
L152       =      Drift('L152'    ,l    =2.847603e+00  )
QD16       = Quadrupole('QD16'    ,l    =4.000000e-01  ,k1   =-1.640586e+00  )
L160       =      Drift('L160'    ,l    =1.168194e+00  )
QF161      = Quadrupole('QF161'   ,l    =4.000000e-01  ,k1   =3.999998e+00  )
L161       =      Drift('L161'    ,l    =5.700426e-01  )
B10        =     Dipole('B10'     ,l    =5.000000e-01  ,angle=1.000000e-02  )
L162       =      Drift('L162'    ,l    =2.200901e+00  )
QF17       = Quadrupole('QF17'    ,l    =4.000000e-01  ,k1   =2.375699e+00  )
L17        =      Drift('L17'     ,l    =2.000263e-01  )
QD18       = Quadrupole('QD18'    ,l    =4.000000e-01  ,k1   =-2.594333e+00  )
L201       =      Drift('L201'    ,l    =2.000000e-01  )
LSCW01     =      Drift('LSCW01'  ,l    =2.000000e-01  )
L20        =      Drift('L20'     ,l    =2.483264e+00  )
QF21       = Quadrupole('QF21'    ,l    =4.000000e-01  ,k1   =2.230491e+00  )
L21        =      Drift('L21'     ,l    =9.078719e-01  )
QD22       = Quadrupole('QD22'    ,l    =4.000000e-01  ,k1   =-1.796830e-01  )
L22        =      Drift('L22'     ,l    =1.456143e+00  )
QF23       = Quadrupole('QF23'    ,l    =4.000000e-01  ,k1   =2.164811e+00  )
L23        =      Drift('L23'     ,l    =1.174351e+00  )
QD24       = Quadrupole('QD24'    ,l    =4.000000e-01  ,k1   =-1.815734e+00  )
L24        =      Drift('L24'     ,l    =3.273376e-01  )
QF25       = Quadrupole('QF25'    ,l    =2.000000e-01  ,k1   =1.609328e+00  )
ARCFODO    =    Lattice('ARCFODO' ,MK0       ,L1        ,QD1       ,L2        ,QF2       ,L31       ,B1        ,L32       ,
                        QD3       ,L4        ,QF4       ,L51       ,L52       ,QD5       ,L6        ,QF6       ,L71       ,
                        B2        ,L72       ,QD7       ,L81       ,LSCY1     ,L82       ,QF8       ,L91       ,B3        ,
                        L92       ,QF9       ,QF9       ,L92       ,B3        ,L91       ,QF8       ,L82       ,LSCY1     ,
                        L81       ,QD7       ,L81       ,LSCY1     ,L10       ,QF10      ,L102      ,B7        ,L103      ,
                        QF102     ,L111      ,B8        ,L112      ,QD11      ,L12       ,QF12      ,L131      ,LSCX1     ,
                        L132      ,QD13      ,L141      ,B5        ,L142      ,QD14      ,QD14      ,L142      ,B6        ,
                        L141      ,QD13      ,L132      ,LSCX1     ,L131      ,QF12      ,L131      ,LSCX1     ,L151      ,
                        QD19      ,L18       ,QF20      ,L19       ,B9        ,L152      ,QD16      ,L160      ,QF161     ,
                        L161      ,B10       ,L162      ,QF17      ,L17       ,QD18      ,L201      ,LSCW01    ,L20       ,
                        QF21      ,L21       ,QD22      ,L22       ,QF23      ,L23       ,QD24      ,L24       ,QF25      ,
                        cell=False)


# lattice = lattice + list(reversed(lattice))
                
# ARCFODO=Lattice('ARCFODO',*lattice,cell=False)


#---OPTIMIZATION---#
# var= (#{'place':MK0[0]  , 'variables': {'beta_x':(0.03,0.04),'beta_y':(0.0005,0.0006)}},
# #       {'place':QD1[0]  , 'variables': {'k1'   : (-7.2+0.5-2, -6.4+2)}},
# #       {'place':L2[0]   , 'variables': {'l'    : (0.15,0.3)}},
# #       {'place':QF2[0]  , 'variables': {'k1'   : (3.01-1, 3.5+2)}},
# #       {'place':L31[0]  , 'variables': {'l'    : (3.5,7.5)}},
# #       {'place':B1[0]   , 'variables': {'angle': (0.00001,0.02)}},
# #       {'place':L32[0]  , 'variables': {'l'    : (0.2,0.5)}},
# #       {'place':QD3[0]  , 'variables': {'k1'   : (-1-2, -0.51-0.2)}},
# #       {'place':L4[0]   , 'variables': {'l'    : (0.2,0.6)}},
# #       {'place':QF4[0]  , 'variables': {'k1'   : (0.0001, 0.1+2)}},
# #       {'place':L51[0]  , 'variables': {'l'    : (0.1,1.5)}},
#       ## telescope point
# #       {'place':L52[0]  , 'variables': {'l'    : (0.5,3)}},
# #       {'place':QD5[0]  , 'variables': {'k1'   : (-8, 0)}},
# # #       {'place':L6[0]   , 'variables': {'l'    : (0.1,1)}},
# #       {'place':QF6[0]  , 'variables': {'k1'   : (0.01, 8)}},
# #       {'place':L71[0]  , 'variables': {'l'    : (0.1,0.9)}},
# #       {'place':B2[0]   , 'variables': {'angle': (0.02,0.05)}},
# #       {'place':L72[0]  , 'variables': {'l'    : (0.1,3)}},
# #       {'place':QD7[0]  , 'variables': {'k1'   : (-8, 0)}},
# #       # vertival chromaticity correction
# #       {'place':L81[0]  , 'variables': {'l'    : (0.1,0.2)}},
# #       {'place':L82[0]  , 'variables': {'l'    : (0.1,2)}},
# #       {'place':QF8[0]  , 'variables': {'k1'   : (0.01, 8)}},
# #       {'place':L91[0]  , 'variables': {'l'    : (0.1,0.9)}},
# #       {'place':B3[0]   , 'variables': {'angle': (0.001,0.05)}},
# #       {'place':L92[0]  , 'variables': {'l'    : (0.1, 3)}},
# #       {'place':QF9[0]  , 'variables': {'k1'   : (0, 8.0)}},
      
# #       {'place':B4[0]   , 'variables': {'angle': (0.001,0.05)}},
      
# #       {'place':L10[0]  , 'variables': {'l'    : (0.1,2)}},
# #       {'place':QF10[0] , 'variables': {'k1'   : (0.01, 8)}},
# #       {'place':L102[0] , 'variables': {'l'    : (0.2,1)}},
# #       {'place':L103[0] , 'variables': {'l'    : (0.1,2)}},
# #       {'place':QF102[0], 'variables': {'k1'   : (0.01, 8)}},
# #       {'place':L111[0]  , 'variables': {'l'    : (0.2,1)}},
# #       {'place':L112[0]  , 'variables': {'l'    : (0.1,2)}},
# #       {'place':QD11[0] , 'variables': {'k1'   : (-4, 0)}},
# #       {'place':L12[0]  , 'variables': {'l'    : (0.1,3)}},
# #       {'place':QF12[0] , 'variables': {'k1'   : (0.01, 8)}},
      
# # #       {'place': L131 ,'variables': {'l'    : {},  },
# #       {'place': L132[0] , 'variables': {'l'    : ( 0.1,  2.0)  }}, 
# #       {'place': QD13[0] , 'variables': {'k1'   : (-4  ,  0  ) }},
# # #       {'place': L141[0] , 'variables': {'l'    : ( 0.1,  2.0)  }}, 
# #       {'place': B5[0]   , 'variables': {'angle': (0.001,  0.05) }}, 
# #       {'place': L142[0] , 'variables': {'l'    : ( 0.1,  2.0)  }}, 
# #       {'place': QD14[0] , 'variables': {'k1'   : (-4  ,  0  ) }}, 
# #       {'place': B6[0]   , 'variables': {'angle': (0.001,  0.05) }},
# # #       {'place': QF15[0] , 'variables': {'k1'   : ( 0  ,  4  ) }}, 
# #       {'place': L17[0]  , 'variables': {'l'    : ( 0.2,  4.0)  }}, 
# #       {'place': QD16[0] , 'variables': {'k1'   : (-4  ,  0  ) }}, 
# #       {'place': L151[0] , 'variables': {'l'    : ( 0.2,  4.2)  }}, 
# #       {'place': B9[0]   , 'variables': {'angle': (-0.01,  0.05) }}, 
# #       {'place': L152[0] , 'variables': {'l'    : ( 0.2,  4.2)  }}, 
# #       {'place': QF161[0] , 'variables': {'k1'   : ( -0  ,  4  ) }},
# #       {'place': QF17[0] , 'variables': {'k1'   : ( -0  ,  4  ) }},
# #       {'place': L160[0] , 'variables': {'l'    : ( 0.2,  4.0-0.5)  }},
# #       {'place': L161[0] , 'variables': {'l'    : ( 0.2,  3.0-0.5)  }},
# # #       {'place': B10[0]   , 'variables': {'angle': (-0.01,  0.05) }},
# # #       {'place': L162[0] , 'variables': {'l'    : ( 0.2,  3.0-0.5)  }},
# #       {'place': QD18[0] , 'variables': {'k1'   : (-4  ,  0  ) }},
# #       {'place': L18[0] , 'variables': {'l'    : ( 0.1,  2.0-0.5)  }},
# #       {'place': QD19[0] , 'variables': {'k1'   : (-4  ,  0  ) }},
# #       {'place': L19[0] , 'variables': {'l'    : ( 0.1,  2.0-0.5)  }},
# #       {'place': QF20[0] , 'variables': {'k1'   : (0  ,  4  ) }},
    
#       {'place': L21[0] , 'variables': {'l'    : ( 0.2,  4.0-0.5)  }},
#       {'place': QF21[0] , 'variables': {'k1'   : (0  ,  4  ) }},
#       {'place': L22[0] , 'variables': {'l'    : ( 0.2,  4.0-0.5)  }},
#       {'place': QD22[0] , 'variables': {'k1'   : (-4  ,  0  ) }},
#       {'place': L23[0] , 'variables': {'l'    : ( 0.2,  4.0-0.5)  }},
#       {'place': QF23[0] , 'variables': {'k1'   : (0  ,  4  ) }},
#       {'place': L20[0] , 'variables': {'l'    : ( 0.2,  4.0-0.5)  }},
#       {'place': QD24[0] , 'variables': {'k1'   : (-4  ,  0  ) }},
#       {'place': L24[0] , 'variables': {'l'    : ( 0.2,  4.0-0.5)  }},
#       {'place': QF25[0] , 'variables': {'k1'   : ( 0  ,  4  ) }},
#        )

# CV= (
# #      {'place': QD1[0]   ,'constraints':{'beta_y':(900,1800)} },
# #      {'place': QF2[0]   ,'constraints':{'beta_x':(50,800),'alpha_y':(0,100)} },
# #      {'place':(MK0[0],QF2[0]) ,'constraints':{'beta_x':(0,700),'beta_y':(0,1800)}},
# #      {'place':(MK0[0], L51[0]) ,'constraints':{"nu_x":(0.5-3e-4,0.5+3e-4),"nu_y":(0.5-3e-4,0.5+3e-4)}},#'alpha_x':0,'alpha_y':0,
    
# #      {'place':(QF2[0], L51[0]) ,'constraints':{'beta_x':(0, 700),'beta_y':(0 , 1200)} },
# # #      {'place': L51[0] ,'constraints':{"beta_x":(0,4),"beta_y":(0,0.05), 'eta_x':(0.08,0.4)}},
    
# #      {'place':(QD1[0], QD7[0]) ,'constraints':{'nu_x':(0.5-2e-2 , 0.5+2e-2 ),'nu_y':(0.5-2e-2 , 0.5+2e-2)} },#
# #      {'place': QD7[0]   ,'constraints':{'beta_y':(300,800), 'eta_x':(0.1+0.01,0.5)} },
# # #      {'place': (QD7[0],QD7[1])   ,'constraints':{'eta_x':0} },
# #      {'place':(QD7[0], QF9[0]) ,'constraints':{'nu_x':(0.25-3e-3, 0.25+3e-3),'nu_y':(0.25-3e-3, 0.25+3e-3)} },#
    
# #      {'place':(QF9[0], QD7[1]) ,'constraints':{'nu_x':(0.25-3e-3, 0.25+3e-3),'nu_y':(0.25-3e-3, 0.25+3e-3)} },
# #      {'place': QD7[1]   ,'constraints':{'beta_y':(300,800), 'eta_x':(0.1+0.01,0.5)} },
# #      {'place': QF12[0]  ,'constraints':{'beta_x':(100,600 ),'beta_y':(0,60 ), 'eta_x':(0.1+0.04,0.5)} },
# #      {'place': (QF12[0], QF12[1]) ,'constraints':{'eta_x':(-0.08,0.15)} },
# #      {'place':(QD3[0],QF12[1]) ,'constraints':{'beta_x':(0,400),'beta_y':(0,800)}},
# #      {'place':(QF2[0], QF12[0]) ,'constraints':{'nu_x':(1.5-3e-2, 1.5+3e-2),'nu_y':(1.5-3e-2 , 1.5+3e-2)} },
    
# #      {'place':(QF12[0], QD14[0]) ,'constraints':{'nu_x':(0.25-3e-2, 0.25+3e-2),'nu_y':(0.25-3e-2 , 0.25+3e-2)} },
# #      {'place': QF12[1]  ,'constraints':{'beta_x':(100,600 ),'beta_y':(0,60 ), 'eta_x':(0.1+0.04,0.5)} },
# #      {'place':(QD14[0], QF12[1]) ,'constraints':{'nu_x':(0.25-3e-2, 0.25+3e-2),'nu_y':(0.25-3e-2 , 0.25+3e-2)} },
# #      {'place':QD14[0] ,'constraints':{'beta_y':(0, 100)} },
# #      {'place':QF12[1] ,'constraints':{'s':(0, 45)} },
# # #      {'place':(MK0[0], QD18[0]) ,'constraints':{'angle': (0.35/2-0.005 , 0.35/2+0.005) } },#
    
# #      {'place':(MK0[0], QD18[0]) ,'constraints':{'nu_x':(2.5+0.5-3e-2, 2.5+0.5+3e-2),'nu_y':(2.75-3e-2 , 2.75+3e-2)}},
# #     ,'s':(0,54.5)} },
# #      {'place':QF12[0] ,'constraints':{'alpha_x':0,'alpha_y':0} },
     
# #      {'place':(QD7[2], QD7[3]) ,'constraints':{'beta_y':(-10, 10)} },#
# #      {'place':QD18[0]   ,'constraints':{'beta_y':(200,800), 'beta_x':(10,100) }},#','eta_x':(-1e-4,1e-4),'etap_x':(-1e-4,1e-4)
# #      {'place':(QD16[0]  ,QF17[0])   ,'constraints':{'beta_y':(0,200),'beta_x':(0,200)} },#'
# #      {'place':(MK0[1]) ,'constraints':{'beta_x': (0.01 , 0.06), 'beta_y':(5e-4 , 8e-4) },'eta_x' :(-1e-6,1e-6), },#
# #      {'place':QD18[0] ,'constraints':{'alpha_y':(-1e-4,1e-4),'alpha_x':(-1e-4,1e-4)}},
#      {'place':QF25[0] ,'constraints':{'alpha_y':(-1e-4,1e-4),'alpha_x':(-1e-4,1e-4),'beta_y':(6,100), 'beta_x':(4,100)}},
#      {'place':QF25[0] ,'constraints':{'nu_x':(3.5+0-3e-2, 3.5+0+1e-2),'nu_y':(3.0+0-3e-2 , 3.0+0+1e-2)}},
#     )

opti = ({'place':QF2[0],'optima':{'chromx':1}},
        {'place':QF2[0],'optima':{'chromy':1}},
# #         {'place':1,'optima':{'emit':1}},
# #         {'place':8,'optima':{'alpha_y':-1}},
# #         {'place':8,'optima':{'alpha_x':-1}},
       )


# ARCFODO['variables'] = Variables(*var)
# ARCFODO['constraints']=Constraints(*CV,tol=1.0e-4)
# ARCFODO['optima'] = Optima(*opti)



from myproblem import MyProblem as Problem
from main import main as main
from cProfile import run
import geatpy as ea
%matplotlib
# problem = Problem(ARCFODO)

Encoding = 'RI'           # 编码方式
NIND=3000
# Field = ea.crtfld(Encoding, problem.varTypes, problem.ranges, problem.borders) # 创建区域描述器
# population = ea.Population(Encoding, Field, NIND) # 实例化种群对象（此时种群还没被初始化，仅仅是完成种群对象的实例化）
# NDSet,pop_trace = main(problem,population,NIND=NIND,MAXGEN=400,drawing=2,CVdrawing=2,Parallel=False)
# ',sort='cumulative')



# run('main(problem,NIND=300,MAXGEN=200,drawing=1,Parallel=True)',sort='cumulative')




In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Oct 10 12:14:10 2020

@author: LT
"""


# ARCFOFO lattice definition
from atpy import *
#---ELEMENTS DIFINE---
MK0 = Marker('MK0',beta_x=0.04,alpha_x=0,gamma_x=25,beta_y=0.0006,alpha_y=0,gamma_y=5000/3)
L1 = Drift('L1',l=0.9)
L2 = Drift('L2',l=0.176560)
QD1 = Quadrupole('QD1',l=0.3,k1=-6.660540)
QF2 = Quadrupole('QF2',l=0.4,k1= 3.525161)
L31 = Drift('L31',l=7.492411)
L32 = Drift('L32',l=0.5)
QD3 = Quadrupole('QD3',l=0.4,k1=-1.299573)
L4 = Drift('L4',l=0.268148)
QF4 = Quadrupole('QF4',l=0.4,k1= 0.0001)
L51 = Drift('L51',l=1.02321)

L52 = Drift('L52',l=0.716178)

QD5 = Quadrupole('QD5',l=0.4,k1=-4.302885)
L6 = Drift('L6',l=0.2)
L61 = Drift('L61',l=1.5)
L62 = Drift('L62',l=1.0)
QF6 = Quadrupole('QF6',l=0.4,k1= 3.609662)
L7 = Drift('L7',l=1.0)
L71 = Drift('L71',l=0.453559)
L72 = Drift('L72',l=1.127190)


QD7 = Quadrupole('QD7',l=0.4,k1=-1.605968)
L8 = Drift('L8',l=0.8)
L81 = Drift('L81',l=0.2)
LSCY1 = Drift('LSCY1',l=0.2)
L82 = Drift('L82',l=0.54302)
QF8 = Quadrupole('QF8',l=0.4,k1= 0.439440)
L91 = Drift('L91',l=0.545217)
L92 = Drift('L92',l=0.913045)

QF10 = Quadrupole('QF10',l=0.4,k1= 0.6)
QF9  = Quadrupole('QF9',l=0.2,k1= 1.263098)
L10= Drift('L10',l=1.623352)
QF10 = Quadrupole('QF10',l=0.4,k1= 1.583713)
L102= Drift('L102',l=0.201362)
L103= Drift('L103',l=0.2)
QF102 = Quadrupole('QF102',l=0.4,k1= 0.6)
L11= Drift('L11',l=1.0)
L111= Drift('L111',l=0.2)
L112= Drift('L112',l=1.0)
QD11 = Quadrupole('QD11',l=0.4,k1= -0.6)
L12= Drift('L12',l=1.0)
QF12 = Quadrupole('QF12',l=0.4,k1= 0.6)

L131=Drift('L131',l=0.1)
LSCX1=Drift('LSCX1',l=0.2)
L132=Drift('L132',l=0.3)
QD13 = Quadrupole('QD13',l=0.4,k1= -0.6)

L141=Drift('L141',l=0.1)
L142=Drift('L142',l=0.3)
QD14 = Quadrupole('QD14',l=0.2,k1= -0.6)
QF15 = Quadrupole('QF15',l=0.4,k1= 0.6)
L15=Drift('L15',l=0.3)
QD16 = Quadrupole('QD16',l=0.4,k1= -0.6)
L161=Drift('L161',l=0.3)
L162=Drift('L162',l=0.3)
QF17 = Quadrupole('QF17',l=0.4,k1= 0.6)
L171=Drift('L171',l=0.3)
L172=Drift('L172',l=0.3)
QD18 = Quadrupole('QD18',l=0.4,k1= -0.6)


B1 = Dipole('B1',l=0.8,angle=0.02)
B2 = Dipole('B2',l=0.5,angle=0.049653)
B3 = Dipole('B3',l=0.5,angle=0.049963)
B4 = Dipole('B4',l=0.5,angle=0.01)
B5 = Dipole('B5',l=0.5,angle=0.01)
B6 = Dipole('B6',l=0.5,angle=0.01)
B7 = Dipole('B7',l=0.5,angle=0.049991)
B8 = Dipole('B8',l=0.5,angle=0.01)

#---CELL DEFINE---#
lattice=[MK0, L1   , QD1  , L2   , QF2  , L31  , B1   , L32  , QD3  , L4   , QF4  ,
         L51  ,
         L52  , QD5  , L6   , QF6  , L71  , B2   , L72  , QD7  , L81  , LSCY1, L82  , 
         QF8  , L91  , B3   , L92  , QF9  , QF9  , L92  , B3   , L91  , QF8  , L82  , LSCY1, L81  , QD7  ,
         L81  , LSCY1, L10  , QF10 , L102 , B7   , L103 , QF102, L111 , B8   , L112 , QD11 , L12  , QF12 , L131 , LSCX1, L132 , QD13 ,
         L141 , B5   , L142 , QD14 , QD14 , L142 , B6   , L141 , QD13 , L132 , LSCX1, L131 , QF12 , 
         L131 , LSCX1,
#          L15  , QD16 , L161 , B7   , L162 , QF17 , L171 , B8   , L172 , QD18 , 
        ]
# lattice = lattice + list(reversed(lattice))
                
ARCFODO=Lattice('ARCFODO',*lattice,cell=False)
# ARCFODO + (-ARCFODO)
#---OPTIMIZATION---#
var= (#{'place':MK0[0]  , 'variables': {'beta_x':(0.03,0.04),'beta_y':(0.0005,0.0006)}},
#       {'place':QD1[0]  , 'variables': {'k1'   : (-7.2+0.5-2, -6.4+2)}},
#       {'place':L2[0]   , 'variables': {'l'    : (0.15,0.3)}},
#       {'place':QF2[0]  , 'variables': {'k1'   : (3.01-1, 3.5+2)}},
#       {'place':L31[0]  , 'variables': {'l'    : (3.5,7.5)}},
#       {'place':B1[0]   , 'variables': {'angle': (0.00001,0.02)}},
#       {'place':L32[0]  , 'variables': {'l'    : (0.2,0.5)}},
#       {'place':QD3[0]  , 'variables': {'k1'   : (-1-2, -0.51-0.2)}},
#       {'place':L4[0]   , 'variables': {'l'    : (0.2,0.6)}},
#       {'place':QF4[0]  , 'variables': {'k1'   : (0.0001, 0.1+2)}},
#       {'place':L51[0]  , 'variables': {'l'    : (0.1,1.5)}},
      ## telescope point
#       {'place':L52[0]  , 'variables': {'l'    : (0.5,3)}},
#       {'place':QD5[0]  , 'variables': {'k1'   : (-8, 0)}},
# #       {'place':L6[0]   , 'variables': {'l'    : (0.1,1)}},
#       {'place':QF6[0]  , 'variables': {'k1'   : (0.01, 8)}},
#       {'place':L71[0]  , 'variables': {'l'    : (0.1,0.9)}},
#       {'place':B2[0]   , 'variables': {'angle': (0.02,0.05)}},
#       {'place':L72[0]  , 'variables': {'l'    : (0.1,3)}},
#       {'place':QD7[0]  , 'variables': {'k1'   : (-8, 0)}},
#       # vertival chromaticity correction
#       {'place':L81[0]  , 'variables': {'l'    : (0.1,0.2)}},
#       {'place':L82[0]  , 'variables': {'l'    : (0.1,2)}},
#       {'place':QF8[0]  , 'variables': {'k1'   : (0.01, 8)}},
#       {'place':L91[0]  , 'variables': {'l'    : (0.1,0.9)}},
#       {'place':B3[0]   , 'variables': {'angle': (0.001,0.05)}},
#       {'place':L92[0]  , 'variables': {'l'    : (0.1, 3)}},
#       {'place':QF9[0]  , 'variables': {'k1'   : (0, 8.0)}},
      
#       {'place':B4[0]   , 'variables': {'angle': (0.001,0.05)}},
      
      {'place':L10[0]  , 'variables': {'l'    : (0.1,2)}},
      {'place':QF10[0] , 'variables': {'k1'   : (0.01, 8)}},
      {'place':L102[0] , 'variables': {'l'    : (0.2,1)}},
      {'place':L103[0] , 'variables': {'l'    : (0.1,2)}},
      {'place':QF102[0], 'variables': {'k1'   : (0.01, 8)}},
      {'place':L111[0]  , 'variables': {'l'    : (0.2,1)}},
      {'place':L112[0]  , 'variables': {'l'    : (0.1,2)}},
      {'place':QD11[0] , 'variables': {'k1'   : (-4, 0)}},
      {'place':L12[0]  , 'variables': {'l'    : (0.1,3)}},
      {'place':QF12[0] , 'variables': {'k1'   : (0.01, 8)}},
      
#       {'place': L131 ,'variables': {'l'    : {},  },
      {'place': L132[0] , 'variables': {'l'    : ( 0.1,  2.0)  }}, 
      {'place': QD13[0] , 'variables': {'k1'   : (-4  ,  0  ) }},
#       {'place': L141[0] , 'variables': {'l'    : ( 0.1,  2.0)  }}, 
      {'place': B5[0]   , 'variables': {'angle': (0.001,  0.05) }}, 
      {'place': L142[0] , 'variables': {'l'    : ( 0.1,  2.0)  }}, 
      {'place': QD14[0] , 'variables': {'k1'   : (-4  ,  0  ) }}, 
      {'place': B6[0]   , 'variables': {'angle': (0.001,  0.05) }},
#       {'place': QF15[0] , 'variables': {'k1'   : ( 0  ,  4  ) }}, 
#       {'place': L15[0]  , 'variables': {'l'    : ( 0.1,  2.0)  }}, 
#       {'place': QD16[0] , 'variables': {'k1'   : (-4  ,  0  ) }}, 
#       {'place': L161[0] , 'variables': {'l'    : ( 0.1,  2.0)  }}, 
      {'place': B7[0]   , 'variables': {'angle': (0.001,  0.05) }}, 
#       {'place': L162[0] , 'variables': {'l'    : ( 0.1,  2.0)  }}, 
#       {'place': QF17[0] , 'variables': {'k1'   : ( 0  ,  4  ) }},
#       {'place': L171[0] , 'variables': {'l'    : ( 0.1,  2.0)  }},
      {'place': B8[0]   , 'variables': {'angle': (0.001,  0.05) }},
#       {'place': L172[0] , 'variables': {'l'    : ( 0.1,  2.0)  }},
#       {'place': QD18[0] , 'variables': {'k1'   : (-4  ,  0  ) }},
       )

CV= (
#      {'place': QD1[0]   ,'constraints':{'beta_y':(900,1800)} },
#      {'place': QF2[0]   ,'constraints':{'beta_x':(50,800),'alpha_y':(0,100)} },
#      {'place':(MK0[0],QF2[0]) ,'constraints':{'beta_x':(0,700),'beta_y':(0,1800)}},
#      {'place':(MK0[0], L51[0]) ,'constraints':{"nu_x":(0.5-3e-4,0.5+3e-4),"nu_y":(0.5-3e-4,0.5+3e-4)}},#'alpha_x':0,'alpha_y':0,
    
#      {'place':(QF2[0], L51[0]) ,'constraints':{'beta_x':(0, 700),'beta_y':(0 , 1200)} },
# #      {'place': L51[0] ,'constraints':{"beta_x":(0,4),"beta_y":(0,0.05), 'eta_x':(0.08,0.4)}},
    
#      {'place':(QD1[0], QD7[0]) ,'constraints':{'nu_x':(0.5-2e-2 , 0.5+2e-2 ),'nu_y':(0.5-2e-2 , 0.5+2e-2)} },#
#      {'place': QD7[0]   ,'constraints':{'beta_y':(300,800), 'eta_x':(0.1+0.01,0.5)} },
# #      {'place': (QD7[0],QD7[1])   ,'constraints':{'eta_x':0} },
#      {'place':(QD7[0], QF9[0]) ,'constraints':{'nu_x':(0.25-3e-3, 0.25+3e-3),'nu_y':(0.25-3e-3, 0.25+3e-3)} },#
    
#      {'place':(QF9[0], QD7[1]) ,'constraints':{'nu_x':(0.25-3e-3, 0.25+3e-3),'nu_y':(0.25-3e-3, 0.25+3e-3)} },
#      {'place': QD7[1]   ,'constraints':{'beta_y':(300,800), 'eta_x':(0.1+0.01,0.5)} },
     {'place': QF12[0]  ,'constraints':{'beta_x':(100,600 ),'beta_y':(0,60 ), 'eta_x':(0.1+0.04,0.5)} },
     {'place': (QF12[0], QF12[1]) ,'constraints':{'eta_x':(-0.08,0.15)} },
     {'place':(QD3[0],QF12[1]) ,'constraints':{'beta_x':(0,400),'beta_y':(0,800)}},
     {'place':(QF2[0], QF12[0]) ,'constraints':{'nu_x':(1.5-3e-2, 1.5+3e-2),'nu_y':(1.5-3e-2 , 1.5+3e-2)} },
    
     {'place':(QF12[0], QD14[0]) ,'constraints':{'nu_x':(0.25-3e-2, 0.25+3e-2),'nu_y':(0.25-3e-2 , 0.25+3e-2)} },
     {'place': QF12[1]  ,'constraints':{'beta_x':(100,600 ),'beta_y':(0,60 ), 'eta_x':(0.1+0.04,0.5)} },
     {'place':(QD14[0], QF12[1]) ,'constraints':{'nu_x':(0.25-3e-2, 0.25+3e-2),'nu_y':(0.25-3e-2 , 0.25+3e-2)} },
     {'place':QD14[0] ,'constraints':{'beta_y':(0, 100)} },
     {'place':QF12[1] ,'constraints':{'s':(0, 45)} },
# #      {'place':(MK0[0], QD18[0]) ,'constraints':{'angle': (0.35/2-0.005 , 0.35/2+0.005) } },#
    
#      {'place':(MK0[0], QD18[0]) ,'constraints':{'nu_x':(2.5-3e-2, 2.5+3e-2),'nu_y':(2.75-3e-2 , 2.75+3e-2)} },
#      {'place':QF12[0] ,'constraints':{'alpha_x':0,'alpha_y':0} },
     
#      {'place':(QD7[2], QD7[3]) ,'constraints':{'beta_y':(-10, 10)} },#
#      {'place':QD18[0]   ,'constraints':{'eta_x':(-1e-4,1e-4),'etap_x':(-1e-4,1e-4)} },#'beta_y':(200,800), 'beta_x':(10,100),
#      {'place':(MK0[1]) ,'constraints':{'beta_x': (0.01 , 0.06), 'beta_y':(5e-4 , 8e-4) },'eta_x' :(-1e-6,1e-6), },#
#      {'place':L51[0] ,'constraints':{'alpha_y':(-1e-4,1e-4),'alpha_x':(-1e-4,1e-4)}},
    )

opti = ({'place':QF2[0],'optima':{'chromx':1}},
        {'place':QF2[0],'optima':{'chromy':1}},
# #         {'place':1,'optima':{'emit':1}},
# #         {'place':8,'optima':{'alpha_y':-1}},
# #         {'place':8,'optima':{'alpha_x':-1}},
       )


ARCFODO['variables'] = Variables(*var)
ARCFODO['constraints']=Constraints(*CV,tol=1.0e-4)
ARCFODO['optima'] = Optima(*opti)



from myproblem import MyProblem as Problem
from main import main as main
from cProfile import run
import geatpy as ea
%matplotlib
problem = Problem(ARCFODO)

Encoding = 'RI'           # 编码方式
NIND=1200
Field = ea.crtfld(Encoding, problem.varTypes, problem.ranges, problem.borders) # 创建区域描述器
population = ea.Population(Encoding, Field, NIND) # 实例化种群对象（此时种群还没被初始化，仅仅是完成种群对象的实例化）
NDSet,pop_trace = main(problem,population,NIND=NIND,MAXGEN=400,drawing=2,CVdrawing=2,Parallel=False)
# ',sort='cumulative')



# run('main(problem,NIND=300,MAXGEN=200,drawing=1,Parallel=True)',sort='cumulative')

In [1]:
#BOKEH plot figure
from bokeh.plotting import figure,show#,output_notebook
import numpy as np
import pandas as pd
from bokeh.models import HoverTool, ColumnDataSource, CustomJS,  CheckboxGroup
from bokeh.layouts import gridplot

import yaml
from bokeh.layouts import column,row
from bokeh.themes import Theme
from bokeh.io import show, output_notebook,output_file

output_notebook()
# output_file('Twiss.html')
# lattice = ARCFODO
lattice = HALF7BA


def pic_h(row):
    if row.element_type.endswith('Dipole'): return 4
    if row.element_type.startswith('Sext'): return 5
    if row.element_type.endswith('Quad'): return 6
#     if row.element_name.startswith('B'): return 5
    return 3

def pic_L(row):
    if row.l < 0.01:
        return 0.01
    else:
        return row.l

def pic_color(row):
    if row.element_type.endswith('Dipole'): return 'green'
    if row.element_type.startswith('Sext'): return 'yellow'
    if row.element_type.endswith('Quadrupole'): 
        if row.k1 >0: return 'red'
        return 'blue'
#     if row.ElementName.startswith('BPM'): return 'magenta'
    return 'white'

def plot_elements():
    df_parms["pic_h"] = df_parms.apply(pic_h, axis = 1)
    df_parms["pic_L"] = df_parms.apply(pic_L, axis = 1)
    df_parms["pic_color"] = df_parms.apply(pic_color, axis = 1)

    all_elements_src = ColumnDataSource(data=dict(
        Index=df_parms.index.values,
        s = df_parms.s0.values,
        L = df_parms.pic_L.values,
        h = df_parms.pic_h.values,
        color = df_parms.pic_color.values,
        Type = df_parms.element_type.values,
        K1 = df_parms.k1.values,
        End = df_parms.s.values
    ))

    p = figure(tools="save,xbox_zoom,xpan,reset", toolbar_location="above",
                plot_width=800, plot_height=110, active_drag="xbox_zoom",
                y_axis_type=None)

    r1 = p.rect('s', 0, width='L', height='h', fill_color='color',
                fill_alpha=0.5, line_width=1.0, line_alpha=0.2,
                line_color='black', source=all_elements_src)

    tips = [
        ('Index', '@Index'),
        ('L','@L'),
        ('K1','@K1'),
        ('Type', '@Type'),
        ('End', '@End')
    ]

    hover = HoverTool(tooltips=tips)
    p.add_tools(hover)

    hover.renderers=[r1]

    p.y_range.start = -7
    p.y_range.end   = +7
    p.xaxis.axis_label='s (m)'

    return p


def sqrt_beta_x(row):
    return np.sqrt(row.beta_x)

def sqrt_beta_y(row):
    return np.sqrt(row.beta_y)


def get_dataframe(line):
#     twisses = [magnet.get_value('twiss') for magnet in problem.line.pyseq]

#     parms = [magnet.get_value('parameters') for magnet in problem.line.pyseq]
    twisses = [magnet.get_value('twiss') for magnet in line['beamline']]

    parms = [magnet.get_value('parameters') for magnet in line['beamline']]

    df_twisses = pd.DataFrame(twisses,columns=['beta_x','alpha_x','gamma_x','beta_y','alpha_y','gamma_y','beta_z','alpha_z',
                                            'gamma_z','nu_x','nu_y','nu_z','eta_x','etap_x','eta_y','etap_y','eta_z','etap_z','chromx','chromy'])
    df_parms = pd.DataFrame(parms,columns=['element_type','l','angle','k1','k2','e1','e2'])
    df_parms['s'] = df_parms['l'].cumsum()

    df_parms['s0'] = df_parms['s'] - df_parms['l']/2# centers of all elements
    df_parms = df_parms[['s','s0','element_type','l','angle','k1','k2','e1','e2']]
    df_twisses['s'] = df_parms.s.copy()
    if np.any(df_twisses[['beta_x','beta_y']]>100):
        df_twisses[['beta_x','beta_y']]=np.sqrt(df_twisses[['beta_x','beta_y']])
#     df_twisses["sqrt_beta_x"] = df_twisses.apply(sqrt_beta_x, axis = 1)
#     df_twisses["sqrt_beta_y"] = df_twisses.apply(sqrt_beta_y, axis = 1)
    return df_parms ,df_twisses

start =0
end = start+1
# ARCFODO.evaluate_lattice()
problem.line.evaluate_lattice()
# NDSet = population[best_gen]
# problem.line.moea_without_CV(NDSet.Phen[start:end,:],NDSet.ObjV[start:end,:],problem.scale,addCV=True)#aimFunc(NDSet[0])
problem.line.get_results(NDSet.Phen[start:end,:],NDSet.ObjV[start:end,:],NDSet.CV[start:end,:],problem.scale)#aimFunc(NDSet[0])
# problem.aimFunc(NDSet)
# problem.line.match(finalpopulation.Phen[start:end,:],finalpopulation.ObjV[start:end,:])#aimFunc(NDSet[0])
df_parms,df_twisses = get_dataframe(lattice)

source = ColumnDataSource(df_twisses)

def bkapp(doc):
    def create_figure(checkbox_group):
        
        p_layout = plot_elements()
        lay_out = []
        if 0 in checkbox_group.active or 1 in checkbox_group.active:
            yaxis_label = 'Beta '
            p = figure(tools="save,xbox_zoom,ybox_zoom,xpan,reset", x_range=p_layout.x_range, toolbar_location="right",
                                 plot_width=800, plot_height=300, active_drag="xbox_zoom")
            if 0 in checkbox_group.active:
                p.line('s','beta_x', color='red', source=source, line_width=2, line_alpha=0.5, legend_label=r"$\beta_x$")
                yaxis_label = yaxis_label + 'x'
            if 1 in checkbox_group.active:
                p.line('s','beta_y', source=source, color='blue', line_width=2, line_alpha=0.5, legend_label=r'$\beta_y$')
                yaxis_label = yaxis_label + ' y'
            p.legend.background_fill_alpha = 0.5
            p.y_range.start = 0; 
            yaxis_label = yaxis_label + ' (m)'

            #p.y_range.end = df[['betax','betay']].values.max() + 0.5

            p.xaxis.major_label_text_alpha = 0
            p.xaxis.major_label_text_font_size = '1pt'

            p.yaxis.axis_label = yaxis_label
            
            lay_out.append([p])
        
        if 2 in checkbox_group.active or 3 in checkbox_group.active:
            yaxis_label = 'Alpha '
            p = figure(tools="save,xbox_zoom,xpan,reset", x_range=p_layout.x_range, toolbar_location="right",
                                 plot_width=800, plot_height=300, active_drag="xbox_zoom")
            if 2 in checkbox_group.active:
                p.line('s','alpha_x', color='red', source=source, line_width=2, line_alpha=0.5, legend_label=r'$\alpha_x$')
                yaxis_label = yaxis_label + 'x'
            if 3 in checkbox_group.active:
                p.line('s','alpha_y', color='blue', source=source, line_width=2, line_alpha=0.5, legend_label=r'$\alpha_y$')
                yaxis_label = yaxis_label + ' y'
            p.legend.background_fill_alpha = 0.5
            
            yaxis_label = yaxis_label + ' (m)'

            #p.y_range.end = df[['betax','betay']].values.max() + 0.5

            p.xaxis.major_label_text_alpha = 0
            p.xaxis.major_label_text_font_size = '1pt'

            p.yaxis.axis_label = yaxis_label
            
            lay_out.append([p])
#             'Beta x,y (m)'

        
        if 4 in checkbox_group.active or 5 in checkbox_group.active:
            yaxis_label = 'eta '
            p = figure(tools="save,xbox_zoom,xpan,reset", x_range=p_layout.x_range, toolbar_location="right",
                                 plot_width=800, plot_height=300, active_drag="xbox_zoom")
            if 4 in checkbox_group.active:
                p.line('s','eta_x', color='red', source=source, line_width=2, line_alpha=0.5, legend_label='$\eta_x$')
                yaxis_label = yaxis_label + 'x'
            if 5 in checkbox_group.active:
                p.line('s','etap_x', source=source, color='blue', line_width=2, line_alpha=0.5, legend_label=r'$\eta^,_x$')
                yaxis_label = yaxis_label + ' x'
            p.legend.background_fill_alpha = 0.5
            
            yaxis_label = yaxis_label + ' (m)'

            #p.y_range.end = df[['betax','betay']].values.max() + 0.5

            p.xaxis.major_label_text_alpha = 0
            p.xaxis.major_label_text_font_size = '1pt'

            p.yaxis.axis_label = yaxis_label
            
            lay_out.append([p])
    
        
        
        if 6 in checkbox_group.active or 7 in checkbox_group.active:
            yaxis_label = 'Nu '
            p = figure(tools="save,xbox_zoom,xpan,reset", x_range=p_layout.x_range, toolbar_location="right",
                                 plot_width=800, plot_height=300, active_drag="xbox_zoom")
            if 6 in checkbox_group.active:
                p.line('s','nu_x', color='red', source=source, line_width=2, line_alpha=0.5, legend_label=r'$\nu_x$')
                yaxis_label = yaxis_label + 'x'
            if 7 in checkbox_group.active:
                p.line('s','nu_y', source=source, color='blue', line_width=2, line_alpha=0.5, legend_label=r'$\nu_y$')
                yaxis_label = yaxis_label + ' y'
            p.legend.background_fill_alpha = 0.5
            p.y_range.start = 0; 
            yaxis_label = yaxis_label + ' (rad)'

            #p.y_range.end = df[['betax','betay']].values.max() + 0.5

            p.xaxis.major_label_text_alpha = 0
            p.xaxis.major_label_text_font_size = '1pt'

            p.yaxis.axis_label = yaxis_label
            
            lay_out.append([p])
        
        lay_out.append([p_layout])
        return gridplot(lay_out, toolbar_location='right')
        
    
    checkbox_group = CheckboxGroup(
        labels=[r"$\beta_x$", r'$\beta_y$' ,r'$\alpha_x$', r'$\alpha_y$', r'$\eta_x$', r"$\eta^,_x$",r"$\nu_x$", r'$\nu_y$'], active=[0,1])
    
    def callback(attr, old, new):
        layout.children[1] = create_figure(checkbox_group)
#         pass
    
    
    checkbox_group.on_change('active', callback)
    
    layout = row(checkbox_group, create_figure(checkbox_group))
    doc.add_root(layout)

%matplotlib inline

show(bkapp)

Loading BokehJS ...

NameError: name 'HALF7BA' is not defined

In [ ]:
#-*-dataframe-*-
df_twisses[['beta_x','alpha_x','beta_y','alpha_y','eta_x','etap_x','nu_x','nu_y','chromx','chromy']][MK0[0]:BA4E[0]+1]
# HALF7BA[['chromx','chromy','emit']]
# HALF7BA['angle']
# ARCFODO[['chromx','chromy','emit']]
# df_twisses[['beta_x','alpha_x','beta_y','alpha_y','eta_x','etap_x','nu_x','nu_y']][MK0[0]:BA4E[0]]
# HALF7BA['rad_integrals']
# HALF7BA[{tuple(range(1,68)):['chromx']}] 
# df_twisses[['chromx','chromy']].to_csv('test.csv')
# np.sum(ARCFODO[{tuple(range(1,69)):['angle']}])
# HALF7BA[{tuple(range(1,35)):['angle']}]
# df_parms.to_csv('STCFIR2.1.csv')
df_parms[MK0[0]:BA4E[0]+1]
# QD18[0]
# NDSet.CV[0,-1]
# from numpy import pi
# 2*pi/30

In [ ]:
import random
# random.randint(1e7)
random.uniform(1e5,1e6)

In [ ]:
#plot figure
from bokeh.plotting import figure,show#,output_notebook
import numpy as np
import pandas as pd
from bokeh.models import HoverTool, ColumnDataSource, CustomJS,  CheckboxGroup
from bokeh.layouts import gridplot

import yaml
from bokeh.layouts import column,row
from bokeh.themes import Theme
from bokeh.io import show, output_notebook,output_file

output_notebook()
# output_file('Twiss.html')
lattice = ARCFODO
# lattice = HALF7BA


def pic_h(row):
    if row.element_type.endswith('Dipole'): return 4
    if row.element_type.startswith('Sext'): return 5
    if row.element_type.endswith('Quad'): return 6
#     if row.element_name.startswith('B'): return 5
    return 3

def pic_L(row):
    if row.l < 0.01:
        return 0.01
    else:
        return row.l

def pic_color(row):
    if row.element_type.endswith('Dipole'): return 'green'
    if row.element_type.startswith('Sext'): return 'yellow'
    if row.element_type.endswith('Quadrupole'): 
        if row.k1 >0: return 'red'
        return 'blue'
#     if row.ElementName.startswith('BPM'): return 'magenta'
    return 'white'

def plot_elements():
    df_parms["pic_h"] = df_parms.apply(pic_h, axis = 1)
    df_parms["pic_L"] = df_parms.apply(pic_L, axis = 1)
    df_parms["pic_color"] = df_parms.apply(pic_color, axis = 1)

    all_elements_src = ColumnDataSource(data=dict(
        Index=df_parms.index.values,
        s = df_parms.s0.values,
        L = df_parms.pic_L.values,
        h = df_parms.pic_h.values,
        color = df_parms.pic_color.values,
        Type = df_parms.element_type.values,
        K1 = df_parms.k1.values,
        End = df_parms.s.values
    ))

    p = figure(tools="save,xbox_zoom,xpan,reset", toolbar_location="above",
                plot_width=800, plot_height=110, active_drag="xbox_zoom",
                y_axis_type=None)

    r1 = p.rect('s', 0, width='L', height='h', fill_color='color',
                fill_alpha=0.5, line_width=1.0, line_alpha=0.2,
                line_color='black', source=all_elements_src)

    tips = [
        ('Index', '@Index'),
        ('L','@L'),
        ('K1','@K1'),
        ('Type', '@Type'),
        ('End', '@End')
    ]

    hover = HoverTool(tooltips=tips)
    p.add_tools(hover)

    hover.renderers=[r1]

    p.y_range.start = -7
    p.y_range.end   = +7
    p.xaxis.axis_label='s (m)'

    return p


def sqrt_beta_x(row):
    return np.sqrt(row.beta_x)

def sqrt_beta_y(row):
    return np.sqrt(row.beta_y)


def get_dataframe(line):
#     twisses = [magnet.get_value('twiss') for magnet in problem.line.pyseq]

#     parms = [magnet.get_value('parameters') for magnet in problem.line.pyseq]
    twisses = [magnet.get_value('twiss') for magnet in line['beamline']]

    parms = [magnet.get_value('parameters') for magnet in line['beamline']]

    df_twisses = pd.DataFrame(twisses,columns=['beta_x','alpha_x','gamma_x','beta_y','alpha_y','gamma_y','beta_z','alpha_z',
                                            'gamma_z','nu_x','nu_y','nu_z','eta_x','etap_x','eta_y','etap_y','eta_z','etap_z','chromx','chromy'])
    df_parms = pd.DataFrame(parms,columns=['element_type','l','angle','k1','k2','e1','e2'])
    df_parms['s'] = df_parms['l'].cumsum()

    df_parms['s0'] = df_parms['s'] - df_parms['l']/2# centers of all elements
    df_parms = df_parms[['s','s0','element_type','l','angle','k1','k2','e1','e2']]
    df_twisses['s'] = df_parms.s.copy()
    if np.any(df_twisses[['beta_x','beta_y']]>100):
        df_twisses[['beta_x','beta_y']]=np.sqrt(df_twisses[['beta_x','beta_y']])
#     df_twisses["sqrt_beta_x"] = df_twisses.apply(sqrt_beta_x, axis = 1)
#     df_twisses["sqrt_beta_y"] = df_twisses.apply(sqrt_beta_y, axis = 1)
    return df_parms ,df_twisses

start =0
end = start+1
# ARCFODO.evaluate_lattice()
# problem.line.evaluate_lattice()
# NDSet = population[best_gen]
# problem.line.moea_without_CV(NDSet.Phen[start:end,:],NDSet.ObjV[start:end,:],problem.scale,addCV=True)#aimFunc(NDSet[0])
problem.line.get_results(NDSet.Phen[start:end,:],NDSet.ObjV[start:end,:],NDSet.CV[start:end,:],problem.scale)#aimFunc(NDSet[0])
# problem.line.match(finalpopulation.Phen[start:end,:],finalpopulation.ObjV[start:end,:])#aimFunc(NDSet[0])
df_parms,df_twisses = get_dataframe(ARCFODO)

source = ColumnDataSource(df_twisses)

def bkapp(doc):
    def create_figure(checkbox_group):
        
        p_layout = plot_elements()
        lay_out = []
        if 0 in checkbox_group.active or 1 in checkbox_group.active:
            yaxis_label = 'Beta '
            p = figure(tools="save,xbox_zoom,ybox_zoom,xpan,reset", x_range=p_layout.x_range, toolbar_location="right",
                                 plot_width=800, plot_height=300, active_drag="xbox_zoom")
            if 0 in checkbox_group.active:
                p.line('s','beta_x', color='red', source=source, line_width=2, line_alpha=0.5, legend_label=r"$\beta_x$")
                yaxis_label = yaxis_label + 'x'
            if 1 in checkbox_group.active:
                p.line('s','beta_y', source=source, color='blue', line_width=2, line_alpha=0.5, legend_label=r'$\beta_y$')
                yaxis_label = yaxis_label + ' y'
            p.legend.background_fill_alpha = 0.5
            p.y_range.start = 0; 
            yaxis_label = yaxis_label + ' (m)'

            #p.y_range.end = df[['betax','betay']].values.max() + 0.5

            p.xaxis.major_label_text_alpha = 0
            p.xaxis.major_label_text_font_size = '1pt'

            p.yaxis.axis_label = yaxis_label
            
            lay_out.append([p])
        
        if 2 in checkbox_group.active or 3 in checkbox_group.active:
            yaxis_label = 'Alpha '
            p = figure(tools="save,xbox_zoom,xpan,reset", x_range=p_layout.x_range, toolbar_location="right",
                                 plot_width=800, plot_height=300, active_drag="xbox_zoom")
            if 2 in checkbox_group.active:
                p.line('s','alpha_x', color='red', source=source, line_width=2, line_alpha=0.5, legend_label=r'$\alpha_x$')
                yaxis_label = yaxis_label + 'x'
            if 3 in checkbox_group.active:
                p.line('s','alpha_y', color='blue', source=source, line_width=2, line_alpha=0.5, legend_label=r'$\alpha_y$')
                yaxis_label = yaxis_label + ' y'
            p.legend.background_fill_alpha = 0.5
            
            yaxis_label = yaxis_label + ' (m)'

            #p.y_range.end = df[['betax','betay']].values.max() + 0.5

            p.xaxis.major_label_text_alpha = 0
            p.xaxis.major_label_text_font_size = '1pt'

            p.yaxis.axis_label = yaxis_label
            
            lay_out.append([p])
#             'Beta x,y (m)'

        
        if 4 in checkbox_group.active or 5 in checkbox_group.active:
            yaxis_label = 'eta '
            p = figure(tools="save,xbox_zoom,xpan,reset", x_range=p_layout.x_range, toolbar_location="right",
                                 plot_width=800, plot_height=300, active_drag="xbox_zoom")
            if 4 in checkbox_group.active:
                p.line('s','eta_x', color='red', source=source, line_width=2, line_alpha=0.5, legend_label='$\eta_x$')
                yaxis_label = yaxis_label + 'x'
            if 5 in checkbox_group.active:
                p.line('s','etap_x', source=source, color='blue', line_width=2, line_alpha=0.5, legend_label=r'$\eta^,_x$')
                yaxis_label = yaxis_label + ' x'
            p.legend.background_fill_alpha = 0.5
            
            yaxis_label = yaxis_label + ' (m)'

            #p.y_range.end = df[['betax','betay']].values.max() + 0.5

            p.xaxis.major_label_text_alpha = 0
            p.xaxis.major_label_text_font_size = '1pt'

            p.yaxis.axis_label = yaxis_label
            
            lay_out.append([p])
    
        
        
        if 6 in checkbox_group.active or 7 in checkbox_group.active:
            yaxis_label = 'Nu '
            p = figure(tools="save,xbox_zoom,xpan,reset", x_range=p_layout.x_range, toolbar_location="right",
                                 plot_width=800, plot_height=300, active_drag="xbox_zoom")
            if 6 in checkbox_group.active:
                p.line('s','nu_x', color='red', source=source, line_width=2, line_alpha=0.5, legend_label=r'$\nu_x$')
                yaxis_label = yaxis_label + 'x'
            if 7 in checkbox_group.active:
                p.line('s','nu_y', source=source, color='blue', line_width=2, line_alpha=0.5, legend_label=r'$\nu_y$')
                yaxis_label = yaxis_label + ' y'
            p.legend.background_fill_alpha = 0.5
            p.y_range.start = 0; 
            yaxis_label = yaxis_label + ' (rad)'

            #p.y_range.end = df[['betax','betay']].values.max() + 0.5

            p.xaxis.major_label_text_alpha = 0
            p.xaxis.major_label_text_font_size = '1pt'

            p.yaxis.axis_label = yaxis_label
            
            lay_out.append([p])
        
        lay_out.append([p_layout])
        return gridplot(lay_out, toolbar_location='right')
        
    
    checkbox_group = CheckboxGroup(
        labels=[r"$\beta_x$", r'$\beta_y$' ,r'$\alpha_x$', r'$\alpha_y$', r'$\eta_x$', r"$\eta^,_x$",r"$\nu_x$", r'$\nu_y$'], active=[0,1])
    
    def callback(attr, old, new):
        layout.children[1] = create_figure(checkbox_group)
#         pass
    
    
    checkbox_group.on_change('active', callback)
    
    layout = row(checkbox_group, create_figure(checkbox_group))
    doc.add_root(layout)
    
%matplotlib inline
show(bkapp)

In [ ]:
%%cython -3 -a --force -c=-fopenmp --link-args=-fopenmp
#cython : boundscheck=False,wraparound=
python.exe -m numpy.f2py -m fortran_ndsort -c C:\Users\LT\.ipython\fortran\_fortran_magic_78078559a3b456ef5d72a2b642efad9e.f90
from cython.parallel cimport parallel,prange
import numpy as np
cimport openmp
cdef double alst[100000000] 
# = np.asarray([1]*100000000,dtype=float)
cdef double barray[100000000]
cdef int i,N=1000
# cdef void** ptr = &alst[0]
def para_test():
    cdef int i,N=100000000
    for i in prange(N,nogil=True,num_threads=6):
        barray[i]=alst[i]
def test():
    cdef int i,N=100000000
    for i in range(N):
        barray[i]=alst[i]

In [ ]:
# PROBLEM
# -*- coding: utf-8 -*-
import geatpy as ea
import numpy as np
class MyProblem(ea.Problem): # 继承Problem父类
    def __init__(self, line):
        self.line = line
        name = 'MyProblem' # 初始化name（函数名称，可以随意设置）
        Dim = len(self.line['variables'].index[0])#1 # 初始化Dim（决策变量维数）
        
        self.M = len(self.line['optima'].index[0])
        maxormins = self.line['optima'].index[4] # 初始化maxormins（目标最小最大化标记列表，1：最小化该目标；-1：最大化该目标）
#         maxormins = np.append(self.line['optima'].index[4], [1])  # 初始化maxormins（目标最小最大化标记列表，1：最小化该目标；-1：最大化该目标）
#         maxormins.append(1)
        if self.line['cell']==True:
            self.NCV=len(self.line['constraints'].index[0])+1
        else:
            self.NCV=len(self.line['constraints'].index[0])
        
        varTypes = [0] * Dim # 初始化varTypes（决策变量的类型，0：实数；1：整数）
        lb = self.line['variables'].bounds[0]  # 决策变量下界
        ub = self.line['variables'].bounds[1]  # 决策变量上界
        lbin = [1] * Dim # 决策变量下边界
        ubin = [1] * Dim # 决策变量上边界
        # 调用父类构造方法完成实例化
        ea.Problem.__init__(self, name, self.M, maxormins, Dim, varTypes, lb, ub, lbin, ubin)
        self.scale = np.array([1]*self.NCV, dtype=float)
#         self.scale[-3] = 1.0e2
#         self.scale[-2] = 1.0e6
#         self.scale[-1] = 1.0e6
    
    def aimFunc(self,pop): # 目标函数
        Vars = pop.Phen # 得到决策变量矩阵
        nind = Vars.shape[0]
        if pop.ObjV is None:
            pop.ObjV = np.zeros((nind,self.M),dtype=float)
#             self.factors = np.arange(MAXGEN)
#             gamma = np.exp( (MAXGEN+1-self.factors)/(-1-self.factors) )
#             gamma = np.sqrt( (self.factors+1)/(MAXGEN+1) )
#             self.factors = 1.0e10*gamma
        self.line.moea_without_CV(Vars, pop.ObjV,self.scale,addCV=True,factor=1.0e4)

In [ ]:
# MAIN
import numpy as np
import geatpy as ea   # import geatpy
# from MyProblem import MyProblem # 导入自定义问题接口

if __name__ == '__main__':
    """================================实例化问题对象==========================="""
    problem = MyProblem(ARCFODO) # 生成问题对象
    """==================================种群设置==============================="""
    Encoding = 'RI'       # 编码方式
    NIND = 10000            # 种群规模
    Field = ea.crtfld(Encoding, problem.varTypes, problem.ranges, problem.borders) # 创建区域描述器
    population = ea.Population(Encoding, Field, NIND) # 实例化种群对象（此时种群还没被初始化，仅仅是完成种群对象的实例化）
    """================================算法参数设置============================="""
    myAlgorithm = ea.soea_DE_currentToBest_1_bin_templet(problem, population) # 实例化一个算法模板对象
    myAlgorithm.MAXGEN = 500 # 最大进化代数
    myAlgorithm.mutOper.F = 0.5 # 差分进化中的参数F
    myAlgorithm.recOper.XOVR = 0.7 # 重组概率
    """===========================调用算法模板进行种群进化======================="""
    [population, obj_trace, var_trace] = myAlgorithm.run() # 执行算法模板
    population.save() # 把最后一代种群的信息保存到文件中
    # 输出结果
    best_gen = np.argmin(problem.maxormins * obj_trace[:, 1]) # 记录最优种群个体是在哪一代
    best_ObjV = obj_trace[best_gen, 1]
    print('最优的目标函数值为：%s'%(best_ObjV))
    print('最优的决策变量值为：')
    for i in range(var_trace.shape[1]):
        print(var_trace[best_gen, i])
    print('有效进化代数：%s'%(obj_trace.shape[0]))
    print('最优的一代是第 %s 代'%(best_gen + 1))
    print('评价次数：%s'%(myAlgorithm.evalsNum))
    print('时间已过 %s 秒'%(myAlgorithm.passTime))

## 7-BA

In [2]:
#STCF 7-BA lattice
from atpy import*

#Element define
MK0 =Marker("MK0")#,

LA1E =Drift("LA1E",l=0.5)
LA2E =Drift("LA2E",l=0.5)
LA3E1 =Drift("LA3E1",l=0.15)
LA3E2 =Drift("LA3E2",l=0.15)
LA4E1 =Drift("LA4E1",l=0.15)
LASCY1E =Drift("LASCY1E",l=0.2)
LA4E2 =Drift("LA4E2",l=0.15)

LA5E1 =Drift("LA5E1",l=0.15)
LASCX1E =Drift("LASCX1E",l=0.2)
LA5E2 =Drift("LA5E2",l=0.15)
# LA5E =Drift("LA5E",l=0.5)
LA6E =Drift("LA6E",l=0.5)
LA7E1 =Drift("LA7E1",l=0.15)
LA7E2 =Drift("LA7E2",l=0.15)
LA8E =Drift("LA8E",l=0.5)
LASCX3E1 =Drift("LASCX3E1",l=0.05)
LASCX3E =Drift("LASCX3E",l=0.2)
LASCX3E2 =Drift("LASCX3E2",l=0.05)
LA9E =Drift("LA9E",l=0.01)
LA9E1 =Drift("LA9E1",l=0.01)
LA9E2 =Drift("LA9E2",l=0.01)
LA9E3 =Drift("LA9E3",l=0.1)
LA9E4 =Drift("LA9E4",l=0.1)
LA9E5 =Drift("LA9E5",l=0.1)
LA10E1 =Drift("LA10E1",l=0.15)
LA10E2 =Drift("LA10E2",l=0.15)
LA11E1 =Drift("LA11E1",l=0.05)
LA11E =Drift("LA11E",l=0.05)
LASCX2E1 =Drift("LASCX2E1",l=0.2)
LASCX2E =Drift("LASCX2E",l=0.2)
LA11E2 =Drift("LA11E2",l=0.05)
LA12E =Drift("LA12E",l=0.15)
LA13E =Drift("LA13E",l=0.1)

BA1E =Dipole("BA1E",l=0.7*2,angle=0.01)
BA2E =Dipole("BA2E",l=1.2,angle=0.01)
BA3E =Dipole("BA3E",l=0.5*2,angle=-0.001)
BA4E =Dipole("BA4E",l=0.5*2,angle=0.01)

BA5E =Dipole("BA5E",l=0.075*2,angle=0.0001)
BA6E =Dipole("BA6E",l=0.075*2,angle=0.0001)
BA7E =Dipole("BA7E",l=0.1*2,angle=0.01)

QA1E =Quadrupole("QA1E",l=0.2,k1=0.5)
QA2E =Quadrupole("QA2E",l=0.2,k1=0.5)
QA3E =Quadrupole("QA3E",l=0.2,k1=0.5)
QA4E =Quadrupole("QA4E",l=0.2,k1=0.5)
QA5E =Quadrupole("QA5E",l=0.2,k1=0.5)
QA6E =Quadrupole("QA6E",l=0.2,k1=0.5)
QA7E =Quadrupole("QA7E",l=0.2,k1=0.5)
QA8E1 =Quadrupole("QA8E1",l=0.2,k1=0.5)
QA8E2 =Quadrupole("QA8E2",l=0.2,k1=0.5)
QA9E =Quadrupole("QA9E",l=0.2,k1=0.5)
QA10E =Quadrupole("QA10E",l=0.2,k1=0.5)
QA11E1 =Quadrupole("QA11E1",l=0.2,k1=0.5)
QA11E2 =Quadrupole("QA11E2",l=0.2,k1=0.5)
QA12E =Quadrupole("QA12E",l=0.2,k1=0.5)

#Beamline define
LATTICE = [ MK0   ,
            LA1E  , QA1E  , LA2E  , QA2E  , LA3E1 ,    BA1E  , LA3E2 , QA3E  ,
            LA4E1 ,LASCY1E, LA4E2 , QA4E  , LA5E1  ,LASCX1E, LA5E1 , QA5E  , LA6E  , 
            QA6E  , LA7E1 , BA2E  , LA7E2 , QA7E   , 
            LA8E  , QA8E1  ,LASCX3E1,LASCX3E,LASCX3E1, QA8E1, LA9E  ,  #
            QA9E  , LA10E1, BA3E  , LA10E2,    QA10E ,LA11E, QA11E1 ,QA11E2, LA12E, QA12E  , LA13E , BA4E  ,# 
          ]# LASCX2E1,LASCX2E, LASCX2E1,
LATTICE=LATTICE + list(reversed(LATTICE))
tws0 = {'betax': 7.36, 'alphax': 0,'gammax': 1.0/7.36, 'betay': 3.63,'alphay': 0,'gammay': 1/3.63}
HALF7BA = Lattice("HALF7BA",*LATTICE,cell=True,**tws0
#                   BA7E  ,    BA6E  , BA5E  , LA10E , QA9E  , LA9E5 ,
#                   BA4E  , LA9E4 , QA9E2 , LA9E3 , QA9E0 ,    LA9E2 , BA3E  , LA9E1 ,
#                   QA8E  , LA8E  , QA7E  , LA7E2 , BA2E  ,    LA7E1 , QA6E  , LA6E  , QA5E  , LA5E  ,
#                   QA4E  , LA4E  , QA3E  , LA3E2 , BA1E  ,    LA3E1 , QA2E  , LA2E  , QA1E  , LA1E ,MK0
                 ) #28 + 1 elements


HALF7BA['variables'] = { 'BA4E[0].angle':{'=':'0.6934/2 -BA1E[0].angle -BA2E[0].angle -BA3E[0].angle'},
                        'QA.+[0].k1':{'>':-3,'<':3},
                        'LA.+E[0].l':{'>':0.05 ,'<':2},
                        'LA1E[0].l':{'>':4.5 ,'<':5.5},
                        'LA.+E.[0].l':{'>':0.05 ,'<':1.5},
                        'MK0[0].betax':{'>':1 ,'<':10},
                        'MK0[0].betay':{'>':1 ,'<':10},
                        'BA[123].+[0].angle':{'>':0.001,'<':0.15}, }
# '0.6934 -BA1E[0].angle -BA2E[0].angle -BA3E[0].angle'
HALF7BA['constraints'] = {#'MIN(MK0[0].betax:MK0[1].betax)':{'<':30},
                          '(MK0[0].emitx)':{'>':0 ,'<':8.0e-9},
#                           'ABS(MK0[0].etax)':{'<':1.0e-3},
                          '1.0e-3*DIM(MK0[0].betax,40)+DIM(ABS(MK0[0].etax),1.0e-3)+ 1.0e-3*DIM(MIN(MK0[0].betax:MK0[1].betax),30)+ 1.0e-3*DIM(MK0[1].circumference,35) ':{'<':0},
#                           'ABS(MK0[1].theta- 0.6934)':{'<':1.0e-4},
                          }

HALF7BA['optima'] = {'ABS(MK0[0].totalchromx) + ABS(MK0[1].totalchromy)':'min',
                          'MK0[0].emitx':'min',}



import geatpy as ea
from myproblem import MyProblem as Problem
from main import main
from cProfile import run
%matplotlib
problem = Problem(HALF7BA)
Encoding = 'RI'           # 编码方式

NIND=1000
# print(HALF7BA['variables'])
# print(HALF7BA['constraints'])
# print(HALF7BA['optima'])

Field = ea.crtfld(Encoding, problem.varTypes, problem.ranges, problem.borders) # 创建区域描述器
population = ea.Population(Encoding, Field, NIND) # 实例化种群对象（此时种群还没被初始化，仅仅是完成种群对象的实例化）

NDSet,pop_trace = main(problem,population,NIND=NIND,MAXGEN=200,drawing=2,CVdrawing=2,Parallel=False,CR=0.85)
# ,maxForgetCount=3500
# run('main(problem,population,NIND=NIND,MAXGEN=100,drawing=2,CVdrawing=2,Parallel=True)',sort='cumulative')


2500.0 0.511 4892.367906066536
Total Variables     :41        , variable  : 40        , covary    :1         
Position   keyword  bound          lower         upper
QA1E       k1       both       -3.00e+00      3.00e+00
QA2E       k1       both       -3.00e+00      3.00e+00
QA3E       k1       both       -3.00e+00      3.00e+00
QA4E       k1       both       -3.00e+00      3.00e+00
QA5E       k1       both       -3.00e+00      3.00e+00
QA6E       k1       both       -3.00e+00      3.00e+00
QA7E       k1       both       -3.00e+00      3.00e+00
QA8E1      k1       both       -3.00e+00      3.00e+00
QA9E       k1       both       -3.00e+00      3.00e+00
QA10E      k1       both       -3.00e+00      3.00e+00
QA11E1     k1       both       -3.00e+00      3.00e+00
QA11E2     k1       both       -3.00e+00      3.00e+00
QA12E      k1       both       -3.00e+00      3.00e+00
LA1E       l        both        5.00e-02      2.00e+00
LA2E       l        both        5.00e-02      2.00e+00
LASCY1E   

In [ ]:
#HALF like 7-BA lattice
from atpy import*

#Element define
MK0 =Marker("MK0",beta_x=7.36, alpha_x=0,gamma_x=1.0/7.36, beta_y=3.63,alpha_y=0,gamma_y=1/3.63)

LA1E =Drift("LA1E",l=0.5)
LA2E =Drift("LA2E",l=0.5)
LA3E1 =Drift("LA3E1",l=0.1)
LA3E2 =Drift("LA3E2",l=0.1)
LA4E =Drift("LA4E",l=0.5)
LA5E =Drift("LA5E",l=0.5)
LA6E =Drift("LA6E",l=0.5)
LA7E1 =Drift("LA7E1",l=0.1)
LA7E2 =Drift("LA7E2",l=0.1)
LA8E =Drift("LA8E",l=0.5)
LA9E1 =Drift("LA9E1",l=0.01)
LA9E2 =Drift("LA9E2",l=0.01)
LA9E3 =Drift("LA9E3",l=0.1)
LA9E4 =Drift("LA9E4",l=0.1)
LA9E5 =Drift("LA9E5",l=0.1)
LA10E =Drift("LA10E",l=0.1)

BA1E =Dipole("BA1E",l=0.7*2,angle=0.01)
BA2E =Dipole("BA2E",l=1.2*2,angle=0.01)
BA3E =Dipole("BA3E",l=0.2*2,angle=-0.001)
BA4E =Dipole("BA4E",l=0.55*2,angle=0.01)
BA5E =Dipole("BA5E",l=0.075*2,angle=0.0001)
BA6E =Dipole("BA6E",l=0.075*2,angle=0.0001)
BA7E =Dipole("BA7E",l=0.1*2,angle=0.01)

QA1E =Quadrupole("QA1E",l=0.2,k1=0.5)
QA2E =Quadrupole("QA2E",l=0.2,k1=0.5)
QA3E =Quadrupole("QA3E",l=0.2,k1=0.5)
QA4E =Quadrupole("QA4E",l=0.2,k1=0.5)
QA5E =Quadrupole("QA5E",l=0.2,k1=0.5)
QA6E =Quadrupole("QA6E",l=0.2,k1=0.5)
QA7E =Quadrupole("QA7E",l=0.2,k1=0.5)
QA8E =Quadrupole("QA8E",l=0.2,k1=0.5)
QA9E =Quadrupole("QA9E",l=0.4,k1=0.5)
QA9E0 =Quadrupole("QA9E0",l=0.2,k1=0.5)
QA9E2 =Quadrupole("QA9E2",l=0.2,k1=0.5)

#Beamline define
LATTICE = [MK0   ,
            LA1E  , QA1E  , LA2E  , QA2E  , LA3E1 ,    BA1E  , LA3E2 , QA3E  , LA4E  , QA4E  ,
            LA5E  , QA5E  , LA6E  , QA6E  , LA7E1 ,    BA2E  , LA7E2 , QA7E  , LA8E  , QA8E  ,
            LA9E1 , BA3E  , LA9E2 , QA9E0 , LA9E3 ,    QA9E2 , LA9E4 , BA4E  , LA9E5 , QA9E  ,
            LA10E , BA5E  , BA6E  , BA7E  , ]
LATTICE=LATTICE + list(reversed(LATTICE))
HALF7BA = Lattice("HALF7BA",*LATTICE,cell=True
#                   MK0   ,
#                   LA1E  , QA1E  , LA2E  , QA2E  , LA3E1 ,    BA1E  , LA3E2 , QA3E  , LA4E  , QA4E  ,
#                   LA5E  , QA5E  , LA6E  , QA6E  , LA7E1 ,    BA2E  , LA7E2 , QA7E  , LA8E  , QA8E  ,
#                   LA9E1 , BA3E  , LA9E2 , QA9E0 , LA9E3 ,    QA9E2 , LA9E4 , BA4E  , LA9E5 , QA9E  ,
#                   LA10E , BA5E  , BA6E  , BA7E  , 
#                   BA7E  ,    BA6E  , BA5E  , LA10E , QA9E  , LA9E5 ,
#                   BA4E  , LA9E4 , QA9E2 , LA9E3 , QA9E0 ,    LA9E2 , BA3E  , LA9E1 ,
#                   QA8E  , LA8E  , QA7E  , LA7E2 , BA2E  ,    LA7E1 , QA6E  , LA6E  , QA5E  , LA5E  ,
#                   QA4E  , LA4E  , QA3E  , LA3E2 , BA1E  ,    LA3E1 , QA2E  , LA2E  , QA1E  , LA1E ,MK0
                 ) #28 + 1 elements

variables = ({'place': 0  , 'variables' : { 'beta_x' : (1 , 10), 'beta_y' : (1 , 10) } },
             {'place': 1  , 'variables' : { 'l'      :  ( 2.1  ,  6.0 ) } },
             {'place': 2  , 'variables' : { 'k1'     :  ( 0.1  ,  15.0 ) } },
             {'place': 3  , 'variables' : { 'l'      :  ( 0.15  ,  1.0 ) } },
             {'place': 4  , 'variables' : { 'k1'     :  (-15.0  , -0.1 ) } },
             {'place': 5  , 'variables' : { 'l'      :  ( 0.1  ,  0.80 ) } },
             {'place': 6  , 'variables' : { 'angle'  :  ( 0.001,  0.05*2 ) } },
             {'place': 7  , 'variables' : { 'l'      :  ( 0.1  ,  0.80 ) } },
             {'place': 8  , 'variables' : { 'k1'     :  (-15.0  ,  -0.10 ) } },
             {'place': 9  , 'variables' : { 'l'      :  ( 0.1  ,  0.80 ) } },
             {'place': 10 , 'variables' : { 'k1'     :  ( 0.10  ,  15.0 ) } },
             {'place': 11 , 'variables' : { 'l'      :  ( 0.1  ,  0.80 ) } },
             {'place': 12 , 'variables' : { 'k1'     :  ( 0.10  ,  15.0 ) } },
             {'place': 13 , 'variables' : { 'l'      :  ( 0.1  ,  0.80 ) } },
             {'place': 14 , 'variables' : { 'k1'     :  (-15.0  ,  -0.10 ) } },
             {'place': 15 , 'variables' : { 'l'      :  ( 0.1  ,  1.0 ) } },
             {'place': 16 , 'variables' : { 'angle'  :  ( 0.001,  0.10 ) } },
             {'place': 17 , 'variables' : { 'l'      :  ( 0.1  ,  1.0 ) } },
             {'place': 18 , 'variables' : { 'k1'     :  (-15.0  ,  -0.10 ) } },
             {'place': 19 , 'variables' : { 'l'      :  ( 0.1  ,  0.50 ) } },
             {'place': 20 , 'variables' : { 'k1'     :  ( 0.10  ,  15.0 ) } },
#              {'place': 21 , 'variables' : { 'l'      :  ( 0.1  ,  0.50 ) } },
             {'place': 22 , 'variables' : { 'angle'  :  (-0.1,  -0.0001 ) } },
#              {'place': 23 , 'variables' : { 'l'      :  ( 0.1  ,  0.40 ) } },
             
             
             {'place': 24 , 'variables' : { 'k1'     :  ( 0.10  ,  15.0 ) } },
             {'place': 25 , 'variables' : { 'l'      :  ( 0.1  ,  0.50 ) } },
             {'place': 26 , 'variables' : { 'k1'     :  ( -10  ,  -0.1 ) } },
#              {'place': 27 , 'variables' : { 'l'      :  ( 0.1  ,  0.40 ) } },
             
             {'place': 24+4 , 'variables' : { 'angle'  :  ( 0.001,  0.05*2 ) } },
             {'place': 25+4 , 'variables' : { 'l'      :  ( 0.1  ,  0.5 ) } },
             {'place': 26+4 , 'variables' : { 'k1'     :  ( 2.10  ,  15.0 ) } },
             {'place': 27+4 , 'variables' : { 'l'      :  ( 0.1  ,  0.5 ) } },
             {'place': 28+4 , 'variables' : { 'angle'  :  ( 0.001,  0.05*2 ) } },
             {'place': 29+4 , 'variables' : { 'angle'  :  ( 0.001,  0.05*2 ) } },
             {'place': 30+4 , 'variables' : { 'angle'  :  ( 0.001,  0.05*2 ) } },
             )
# F,D,B,D,F ,F,D,B,D,F,B,B,F,B

constraints = (#{'place' : 30, 'constraints' : {'alpha_x' : 0 , 'alpha_y' : 0 } },
#                {'place' : 28, 'constraints' : {'alpha_x' : 0 , 'alpha_y' : 0, 'etap_x' : 0, 'eta_x' : 0, 'beta_x': (0, 1)} },
               {'place' : (1,60+8), 'constraints' : {'beta_x' : (0,45)} },
#                {'place' : (1,28), 'constraints' : {'beta_x' : (0,10), 'beta_y' : (0,20)} },
#                {'place' : 30+4, 'constraints' : { 'eta_x' : (-2e-3,2e-3)} },# 'etap_x' : (-1.0e-9,1.0e-9),
               {'place' : (1,60+8), 'constraints' : { 'beta_y' : (0,45)} },
#                {'place' : 30+4, 'constraints' : { 'beta_x': (0, 2)} },
               {'place' : 1, 'constraints' : {'emit' : (0.0,2.0e-9)} },
               {'place' : 60+8, 'constraints' : {'eta_x': (-1.0e-4,1.0e-4) }},#, 'etap_x' : (-1.0e-9,1.0e-9)
#                {'place' : 15, 'constraints' : {'eta_x' : (0    , 0.02)} },
#                {'place' : 10, 'constraints' : {'eta_x' : (0.04 , 0.10)} },
#                {'place' : 1, 'constraints' : {'chromx' : (0.0 , 6.0) , 'chromy' : (0.0 , 6.0)} },#
               {'place' : (0,68), 'constraints' : {'angle':(0.5547,0.5548)} },
#                {'place' : (0,68), 'constraints' : {'angle':(0.20,0.21)}},
#                {'place' : (12,64), 'constraints' : {'nu_x':(1.5-0.02,1.5+0.02), 'nu_y':(1.0-0.02,1.0+0.02)}},
#                {'place' : , 'constraints' : {} },
#                {'place' : , 'constraints' : {} },
#                {'place' : , 'constraints' : {} },
#                {'place' : , 'constraints' : {} },
#                {'place' : , 'constraints' : {} },
#                {'place' : , 'constraints' : {} },
              )

optima = ({'place' : 1, 'optima' : {'chromx' : 1}},
          {'place' : 1, 'optima' : {'chromy' : 1}},
         )

HALF7BA['variables'] = Variables(*variables)

HALF7BA['constraints'] = Constraints(*constraints,tol=1.0e-10)

HALF7BA['optima'] = Optima(*optima)



import geatpy as ea
from myproblem import MyProblem as Problem
from main import main
from cProfile import run
%matplotlib
problem = Problem(HALF7BA)
Encoding = 'RI'           # 编码方式

NIND=600

Field = ea.crtfld(Encoding, problem.varTypes, problem.ranges, problem.borders) # 创建区域描述器
population = ea.Population(Encoding, Field, NIND) # 实例化种群对象（此时种群还没被初始化，仅仅是完成种群对象的实例化）

NDSet,pop_trace = main(problem,population,NIND=NIND,MAXGEN=800,drawing=2,CVdrawing=2,Parallel=True,CR=0.8)

# run('main(problem,population,NIND=NIND,MAXGEN=100,drawing=2,CVdrawing=2,Parallel=True)',sort='cumulative')


## make plot

In [2]:
#-*- PLOT WITH MATPLOTLIB -*-
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
import pandas as pd


def pic_h(row):
    if row.element_type.endswith('Dipole'): return 4
    if row.element_type.startswith('Sext'): return 5
    if row.element_type.endswith('Quad'): return 6
#     if row.element_name.startswith('B'): return 5
    return 3


def pic_L(row):
    if row.l < 0.01:
        return 0.01
    else:
        return row.l


def pic_color(row):
    if row.element_type.endswith('Dipole'): return 'green'
    if row.element_type.startswith('Sext'): return 'yellow'
    if row.element_type.endswith('Quadrupole'): 
        if row.k1 >0: return 'red'
        return 'blue'
#     if row.ElementName.startswith('BPM'): return 'magenta'
    return 'white'

    
def plot_elements(dframe,ax):
    dframe["pic_h"] = dframe.apply(pic_h, axis = 1)
    dframe["pic_L"] = dframe.apply(pic_L, axis = 1)
    dframe["pic_color"] = dframe.apply(pic_color, axis = 1)
    for index,row in dframe.iterrows():
        ax.add_patch(Rectangle((row.s-row.l,-0.5*row.pic_h), row.pic_L, row.pic_h, fill=True, facecolor=row.pic_color,alpha=0.5,edgecolor='black') )

        
def sqrt_beta_x(row):
    return np.sqrt(row.betax)

def sqrt_beta_y(row):
    return np.sqrt(row.betay)


def get_dataframe(line):
    tws = [key for key in line['tws'].keys()]
    kwd = ['element_type']+[key for key in line['kwd'].keys()]
    
    twisses = line[{tuple(range(len(line['beamline'])) ):tws}]#[magnet.get_value('twiss') for magnet in line['beamline']]

    parms = [elem['keywords'] for elem in line['beamline']]#[magnet.get_value('parameters') for magnet in line['beamline']]

    df_twisses = pd.DataFrame(twisses,columns=tws)
    
    df_parms = pd.DataFrame(parms,columns=kwd)
    
    df_parms['s'] = df_parms['l'].cumsum()
    df_twisses['name'] = [magnet['name'] for magnet in line['beamline']]
    
    df_twisses['s'] = df_parms.s.copy()
    df_parms['name'] = df_twisses.name.copy()
    
    df_parms = df_parms[['name','element_type','s','l','angle','k1','k2','e1','e2']]
    df_twisses = df_twisses[['name','s','betax','alphax','betay','alphay','etax','etapx',
                             'nux','nuy','chromx','chromy']]
    if np.any(df_twisses[['betax','betay']]>100):
#         df_twisses[['beta_x','beta_y']]=np.sqrt(df_twisses[['beta_x','beta_y']])
        df_twisses["sqrt_beta_x"] = df_twisses.apply(sqrt_beta_x, axis = 1)
        df_twisses["sqrt_beta_y"] = df_twisses.apply(sqrt_beta_y, axis = 1)
    return df_parms ,df_twisses

def main(line,filename=None):
    #get dataframe
    df_parms,df_twisses = get_dataframe(line)
    #make plot
    fig,ax = plt.subplots(2,1,figsize=(16,9),sharex=True, gridspec_kw={'height_ratios':[8,1]})
    ax1=ax[0]
    ax2=ax[1]
    df_twisses.plot(ax=ax1,x='s',y=['betax','betay'],style=['-r','-b'])
    df_twisses.plot(ax=ax1,x='s',y=['etax'],secondary_y=True,style=['--c'])
    #plot_elements
    plot_elements(df_parms,ax2)
    ax2.set_ylim(-5,5)
    if filename:
        plt.savefig(filename,format='png')
        plt.close()
    else:
        plt.show()
    return df_twisses,df_parms

%matplotlib inline
save = False
# save = True
# print(NDSet.Phen.shape[0])
nfig = 1#*NDSet.Phen.shape[0]
for i in range(nfig):
    start =i
    end = start+1
    # problem.line.evaluate_lattice()
    filename='./STCFpicture/STCFMBA{:0>4}.png'.format(i+1) if save else None
#     problem.aimFunc(NDSet)
    problem.line.get_results(NDSet.Phen[start:end,:],NDSet.ObjV[start:end,:],NDSet.CV[start:end,:])
    #aimFunc(NDSet[0])
    df_twiss,df_parms = main(problem.line,filename)
problem.line[['totalchromx','totalchromy','emitx']]
# problem.line[{MK0[-1]:'theta'}]

NameError: name 'problem' is not defined

In [ ]:
filename='STCFMBA03'
problem.line.write2file(filename+'.sad')

problem.line.write2py(filename+'.py')

problem.line.write2madx(filename+'.beamline')

In [3]:
df_twiss[MK0[0]:BA4E[0]+1]
df_parms[['name','s','element_type','l','angle','k1']][MK0[0]:BA4E[0]+1]

NameError: name 'df_twiss' is not defined

In [6]:
dic = {1:[1,2,3,4,5]}
# dic.values().next()
#     print(i)
len([vary[2] for index,value in enumerate(HALF7BA['variables']) for vary in value.values()  ])
population.Phen.shape
# tws_list = ['betax','alphax','betay','alphay','etax','etapx','nux','nuy','chromx','chromy']
tws_list = ['R11','R22','R33','R44']
# print([HALF7BA[i] for i in tws_list])
print(HALF7BA[{MK0[1]:tws_list}])
#     print()

[[ 0.47881132  0.47881132 -0.01759658 -0.01759658]]
